# Chart Analysis Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB


################################################################################
## Chart Stuff
################################################################################
from artistIgnores import getArtistIgnores
from billboardData import billboardData
from top40Data import top40Data
from chartArtistAlbumData import chartArtistAlbumData


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-12-12 16:05:58.476880


# Top40 Data

In [8]:
%load_ext autoreload
%autoreload
from top40Data import top40Data
td = top40Data(minYear=1, maxYear=2021)
td.setChartUsage(rank=[0])
td.setFullChartData()
td.setArtistAlbumData()
td.saveArtistAlbumData()
td.saveFullChartData()

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  Getting Chart For usa
  Using 2 Charts
  Using 2 Charts
  Using Charts (None): ['USA Singles Top 40', 'USA Albums']
Found 49 files.
==> USA Singles Top 40                      	1755
==> USA Albums                              	2926
Renamed 0 artists
Most Common Artists:
Saving 2926 Artist Album Data to currentTop40ArtistAlbumData.p
Saving data to currentTop40ArtistAlbumData.p
  --> This file is 149.7kB.
Saved data to currentTop40ArtistAlbumData.p
  --> This file is 149.7kB.
Saving 2926 Full Artist Data
Saving data to currentTop40FullChartArtistAlbumData.p
  --> This file is 489.6kB.
Saved data to currentTop40FullChartArtistAlbumData.p
  --> This file is 489.6kB.
Current Time is Sat Dec 12, 2020 16:09:43 for Last Run


# Rename Input Data

In [9]:
manDB      = masterArtistNameDB("main", init=False)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 6537 artist keys.
  There are currently 7564 renamed artist keys.


# Load Chart Analysis

In [3]:
#mcm    = matchChartMusic(mdb)
recreate = False
if recreate:
    maindb = mainDB(mdb=None, create=True, debug=True)
    maindb.setDBArtists(recreate=False)
    maindb = mainDB(mdb=None, create=False, debug=False)
    maindb.loadDBDataMap()
else:
    maindb = mainDB(mdb=None, create=False, debug=False)
    maindb.loadDBDataMap()    
_, _ = clock("Last Run")

=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 755094 ID -> Name entries
    Found 688711 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =========================
Summary Statistics For DB: AllMusic
    Using Known Artists: False
    Found 74453 ID -> Name entries
    Found 67822 

# Analyze Data

In [4]:
singleArtistAlbumData = {}
manyArtistAlbumData   = {}

chartType = "Top40"
cad = chartArtistAlbumData(chartType)
cad.createIndivArtistAlbumData()
singleArtistAlbumData[chartType], manyArtistAlbumData[chartType] = cad.getArtistAlbumData()

Found 62 files.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 6537 artist keys.
  There are currently 7564 renamed artist keys.
There are 5035 unique artists in artist albums data
There are 4984 newly unique artists in artist albums data
Renamed 419 artists
There are 4984   unique artist entries in the artist albums data
There are 4984   single artist entries in the artist albums data
There are 0      multi artist entries in the artist albums data
There are 4984   solo artist entries in the artist albums data
There are 0      many artist entries in the artist albums data


### Initialize MusicMaps

In [6]:
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial
import time


def getThresholds(minAlbums=None):
    thresholds = {}
    thresholds[1000] = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 10.0}
    thresholds[500]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 5.0}
    thresholds[200]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 2.5}
    thresholds[100]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(100/8), 'score': 1.5}
    thresholds[50]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(50/8), 'score': 1.5}
    thresholds[20]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 3, 'score': 1.5}
    thresholds[10]   = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
    thresholds[5]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
    thresholds[3]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
    thresholds[2]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 2.0}
    thresholds[1]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 0.9}

    if minAlbums is None:
        return thresholds
    return thresholds[minAlbums]



def getThresholdsWithoutAlbums(cutoff=0.9):
    return {'numArtistName': 5, 'artistNameCutoff': cutoff, 'artistAlbumCutoff': None, 'numArtistAlbums': None, 'score': None}


def matchItAll(mdbmaps, thresholds, useAlbums=True, mdbmc=None, db=None, toMatch=None):
    if toMatch is None:
        toMatch   = mdbmc.getDataToMatch(db, maxValues=10000, maxAlbums=50000)

    if len(toMatch) == 0:
        return

    num_processes = 3
    if useAlbums is True:
        func = matchDBArtistWithAlbums
    else:
        func = matchDBArtistWithoutAlbums
    pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
    #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
    dbName = list(toMatch.keys())[0]
    copyMapData(mdbmaps[dbName])

    argument_list = toMatch[dbName]
    if len(argument_list) == 0:
        return
    print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
    result_list = multiProc(func=pfunc, argument_list=argument_list,
                                           num_processes=num_processes)


    start, cmt = clock("Saving...")
    saveMapData(mdbmaps[dbName], result_list)
    elapsed(start, cmt)
    print("\nSleeping for 10 seconds...\n")
    #sleep(10)
    
    #mdbmc.matchMutualMaps()
    
    
def findMissingArtists(mdbmap, maindb):
    toget = {}
    ###################################################################################################
    # Diagnostics
    ###################################################################################################
    for primaryKey,artistData in mdbmap.items():
        for db,dbID in artistData.getDict().items():
            if dbID is not None:
                secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
                if secondaryArtistName is None:
                    if toget.get(db) is None:
                        toget[db] = []
                    toget[db].append(primaryArtistName)
                    continue    
                    
                    
           
def addDiscogs(mdbmaps, chartType, amID, dbID):
    if dbID is None:
        return
    mdbmaps[chartType].addArtistDataByKey(amID, "Discogs", dbID)

def addAllMusic(mdbmaps, chartType, amID, dbID):
    if dbID is None:
        return
    dbID = dbID[2:]
    mdbmaps[chartType].addArtistDataByKey(amID, "AllMusic", dbID)

def addMusicBrainz(mdbmaps, chartType, amID, url):
    if dbID is None:
        return
    dbID = maindb.getArtistDBIDFromUtil("MusicBrainz", url)
    mdbmaps[chartType].addArtistDataByKey(amID, "MusicBrainz", dbID)

def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = artistData["ArtistAlbums"]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def matchDBArtistWithoutAlbums(item, *args, **kwargs):
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = None
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score']),
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = artistData["ArtistAlbums"]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm


def copyMapData(mdbmap):
    mdbmap.saveCopy()

def saveMapData(mdbmap, result_list):
    for item in result_list:
        primaryKey = item[0] 
        artistName = item[1]
        artistID   = item[2]
        mcs        = item[3]
        mdbmap.addArtistByKey(primaryKey, artistName=artistName, artistID=artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistDataByKey(primaryKey, db, matchID)

    mdbmap.save()

# Create Master Match Class

In [7]:
mdbmaps = {}
mdbmaps[chartType] = musicDBMap(chartType, init=True, copy=False)
mdbmc = masterDBMatchClass(maindb, mdbmaps)
def reMatch(mdbmaps=mdbmaps, mdbmc=mdbmc):
    matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} 
                 for artistName,artistAlbums in singleArtistAlbumData[chartType].items()}
    mdbmc.setDBMatchData(chartType, matchData)
reMatch()
#mbDF = mdbmc.getMasterDF(chartType)

  Loaded 0 previously matched entries
Loading Artist Names
  Setting matchData for Top40


## Match Using Names And Albums

In [ ]:
iterItems = {20: {"Max": 50, "Vals": 100}}
iterItems.update({10: {"Max": 20, "Vals": 200}})
iterItems.update({50: {"Max": 10000, "Vals": 50}})
iterItems.update({5: {"Max": 10, "Vals": 250}})
iterItems.update({3: {"Max": 5, "Vals": 500}})
iterItems.update({2: {"Max": 3, "Vals": 500}})
#iterItems.update({1: {"Max": 2, "Vals": 500}})
for minAlbums,minAlbumsData in iterItems.items():
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=["chartIgnores.yaml"])
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
        
reMatch()
print("Done.\n\n")

## Match Only Using Names

In [ ]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=["chartIgnores.yaml"])
len(toMatch[chartType])

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.975), toMatch=toMatch, useAlbums=False)

In [ ]:
toMatch

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.9), toMatch=toMatch, useAlbums=False)

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.60), toMatch=toMatch, useAlbums=False)

In [ ]:
for item in toMatch[chartType]:
    if True:
        print("add{0}(mdbmaps, {1}, {2}, {3})   ### {4}".format("AllMusic", "\"{0}\"".format(chartType), "\"{0}\"".format(item[0]), "\"{0}\"".format(""), item[1]["ArtistName"]))
        print("add{0}(mdbmaps, {1}, {2}, {3})    ### {4}".format("Discogs", "\"{0}\"".format(chartType), "\"{0}\"".format(item[0]), "\"{0}\"".format(""), item[1]["ArtistName"]))
        albums = singleArtistAlbumData[chartType][item[1]["ArtistName"]]    
        print("### {0}\n".format("  ,  ".join(albums)))
    else:
        print("ignores.append(\"{0}\")".format(item[1]["ArtistName"]))
        if " & " in item[1]["ArtistName"]:
            continue
            print("ignores.append(\"{0}\")".format(item[1]["ArtistName"]))

In [ ]:
addAllMusic(mdbmaps, "Top40", "6cf1242f0b070c5e288603f372eae127", "mn0001815067")   ### Love INC.
addDiscogs(mdbmaps, "Top40", "6cf1242f0b070c5e288603f372eae127", "81815")    ### Love INC.
### You're A Superstar  ,  Broken Bones  ,  Into The Night

addAllMusic(mdbmaps, "Top40", "9fa987eeb76b93fec3bb53b9c3b52901", "mn0000099307")   ### Inme
addDiscogs(mdbmaps, "Top40", "9fa987eeb76b93fec3bb53b9c3b52901", "255822")    ### Inme
### Crushed Like Fruit  ,  7 Weeks  ,  So You Know  ,  Overgrown Eden

addAllMusic(mdbmaps, "Top40", "0bd576bd49428fdb0094e06134c43753", "mn0000580166")   ### Wildhearts
addDiscogs(mdbmaps, "Top40", "0bd576bd49428fdb0094e06134c43753", "293211")    ### Wildhearts
### Stormy In The North Karma In The South  ,  So Into You  ,  Top Of The World

addAllMusic(mdbmaps, "Top40", "b2f5a8435e1c05561949b1cceb7af9c2", "mn0000753347")   ### SNAP
addDiscogs(mdbmaps, "Top40", "b2f5a8435e1c05561949b1cceb7af9c2", "7698")    ### SNAP
### Rhythm Is A Dancer  ,  The Power Of Bhangra 2003

addAllMusic(mdbmaps, "Top40", "86058aed9a9a3acf13a9a98a36b7e7b1", "mn0000311143")   ### Louise
addDiscogs(mdbmaps, "Top40", "86058aed9a9a3acf13a9a98a36b7e7b1", "27672")    ### Louise
### Pandora's Kiss  ,  Changing Faces - The Best Of

addAllMusic(mdbmaps, "Top40", "2a71b96cf0aaacf811c9c2fb1edda234", "mn0000473645")   ### Stills
addDiscogs(mdbmaps, "Top40", "2a71b96cf0aaacf811c9c2fb1edda234", "189130")    ### Stills
### Lola Stars And Stripes

addAllMusic(mdbmaps, "Top40", "b42f63df6c9a68b91cae68ba199b41b1", "mn0000321077")   ### Rachel Mcfarlane
addDiscogs(mdbmaps, "Top40", "b42f63df6c9a68b91cae68ba199b41b1", "35251")    ### Rachel Mcfarlane
### Lover (all Around The World)

addAllMusic(mdbmaps, "Top40", "36317e415b15ef7e932a10cde918bce2", "mn0000861319")   ### Bratz Rock Angelz
addDiscogs(mdbmaps, "Top40", "36317e415b15ef7e932a10cde918bce2", "152451")    ### Bratz Rock Angelz
### So Good

addAllMusic(mdbmaps, "Top40", "314a62e246dcc05bd9d691769d044ee9", "mn0000103025")   ### Crimea
addDiscogs(mdbmaps, "Top40", "314a62e246dcc05bd9d691769d044ee9", "383955")    ### Crimea
### Lottery Winners On Acid

addDiscogs(mdbmaps, "Top40", "82a2f4678b32959df0f20a25f28e36be", "111926")    ### MVP
### Bounce Shake Move Stop

ignores.append("Lahayna")

addAllMusic(mdbmaps, "Top40", "93331e6e7a6e4e93a6f2b5f8bccc7489", "mn0000996141")   ### Ting Tings
addDiscogs(mdbmaps, "Top40", "93331e6e7a6e4e93a6f2b5f8bccc7489", "844278")    ### Ting Tings
### That's Not My Name  ,  Great DJ  ,  Shut Up And Let Me Go  ,  Be The One  ,  Hands  ,  We Started Nothing

addAllMusic(mdbmaps, "Top40", "d903f81881bc4a95c01a1e171682e1c6", "mn0002144795")   ### Yeah You's
addDiscogs(mdbmaps, "Top40", "d903f81881bc4a95c01a1e171682e1c6", "1496557")    ### Yeah You's
### 15 Minutes

addAllMusic(mdbmaps, "Top40", "2ca2c91d43bafdb11540171f04a62c7f", "mn0001802484")   ### Naturi Naughton
addDiscogs(mdbmaps, "Top40", "2ca2c91d43bafdb11540171f04a62c7f", "1525573")    ### Naturi Naughton
### Fame

addAllMusic(mdbmaps, "Top40", "a9593dadc42113f962eb54a99bca2653", "mn0000341718")   ### Jessica Wright
addDiscogs(mdbmaps, "Top40", "a9593dadc42113f962eb54a99bca2653", "3251933")    ### Jessica Wright
### Dance All Night

ignores.append("Dapper Laughs")
### Proper Moist

addDiscogs(mdbmaps, "Top40", "af3bd1548bef61d78989419ab8622b2d", "3287938")    ### Faul
### Changes

addAllMusic(mdbmaps, "Top40", "24b83f753fc60e4c0af78c3364fe1867", "mn0000386829")   ### Matthew Good Band
addDiscogs(mdbmaps, "Top40", "24b83f753fc60e4c0af78c3364fe1867", "272467")    ### Matthew Good Band
### Raygun  ,  Hello Time Bomb  ,  In A World Called Catastrophe

addAllMusic(mdbmaps, "Top40", "a23287318c26ab2edc1fdb7ed4aa04a9", "mn0001543359")   ### Dufort Jean Rene
addDiscogs(mdbmaps, "Top40", "a23287318c26ab2edc1fdb7ed4aa04a9", "3679179")    ### Dufort Jean Rene
### Chu Pas Plus Pourri Qu'un Aut

addAllMusic(mdbmaps, "Top40", "d4ee95268998aa66f8d843bcee2a7f96", "mn0001990795")   ### A.g.
### Let Me See Something

addAllMusic(mdbmaps, "Top40", "9834631d26effa5b226758b72b4ceb30", "mn0001546603")   ### J-Mizz
### Strong Arm

addAllMusic(mdbmaps, "Top40", "5f1397bd26552699c1410b24267bb136", "mn0002182169")   ### Bettina
### She Is

addAllMusic(mdbmaps, "Top40", "11d1d51d24c031a3fbe240d56ff94170", "mn0003142748")   ### Partynextdoor
addDiscogs(mdbmaps, "Top40", "11d1d51d24c031a3fbe240d56ff94170", "3554300")    ### Partynextdoor
### Partynextdoor Two  ,  Partynextdoor 3 (P3)

mdbmaps[chartType].save()

# Find Near Artist Name Matches

In [ ]:
from difflib import SequenceMatcher


ignores     = getFile("chartIgnores.yaml")
dbRenames   = {} #getFile("relDBRenames2.yaml")
savedDBName = chartType

toget = {}

for primaryKey,artistName in mdbmaps[savedDBName].getArtists().items():
    primaryArtistName = artistName
    if primaryArtistName.find("Star Academy") != -1:
        continue
    if primaryArtistName in ignores:
        continue
    if primaryArtistName in ignores:
        continue
    if sum([x in primaryArtistName for x in [" featuring ", " Featuring "]]) > 0:
        ignores.append(primaryArtistName)
        continue
    artistData = mdbmaps[savedDBName].getArtistDataByKey(primaryKey)
    for db,dbID in artistData.getDict().items():
        if db in ["DatPiff", "MetalStorm", "LastFM", "CDandLP", "AceBootlegs", "RockCorner", "MusicStack", "RateYourMusic"]:
            continue
        if dbID is not None:
            secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
            if secondaryArtistName is None:
                if toget.get(db) is None:
                    toget[db] = []
                toget[db].append(primaryArtistName)
                continue
            if secondaryArtistName in ignores:
                continue
            s = SequenceMatcher(None, primaryArtistName, secondaryArtistName)
            ratio = s.ratio()
            maxRatio = 1.0
            if ratio >= maxRatio:
                continue
                
            if ratio >= maxRatio-1.0:
                if dbRenames.get(secondaryArtistName) is not None:
                    if dbRenames[secondaryArtistName] == primaryArtistName:
                        continue
                    else:
                        if secondaryArtistName.find("Star Academy") != -1:
                            del dbRenames[secondaryArtistName]
                            continue
                        if secondaryArtistName.find("Morning Musume") != -1:
                            del dbRenames[secondaryArtistName]
                            continue
                        if False:
                            if secondaryArtistName == "Wild Billy Childish and the Musicians of the British Empire":
                                del dbRenames[secondaryArtistName]
                                continue                            
                            if secondaryArtistName == "Wild Billy Childish & The Musicians Of The British Empire":
                                del dbRenames[secondaryArtistName]
                                continue                            
                            if secondaryArtistName == "Glenn Hughes":
                                del dbRenames[secondaryArtistName]
                                continue                          
                            if secondaryArtistName == "Formula V" or secondaryArtistName == "Formula 3":
                                del dbRenames[secondaryArtistName]
                                continue                          
                        #raise ValueError("Multi Values For {0}: [{1} , {2}]".format(secondaryArtistName, primaryArtistName, dbRenames[secondaryArtistName]))
                #dbRenames[secondaryArtistName] = primaryArtistName
                dbRenames[primaryArtistName] = secondaryArtistName
                print("[{0: <30} {1: <4} {2: >30}] \t --> ({3}) {4} / {5} ({6})".format(primaryArtistName,round(ratio,2),secondaryArtistName,savedDBName,primaryKey,dbID,db))
                
            
print("Done.")

In [ ]:
for artistName,primaryName in dbRenames.items():
    break
    if " & " in primaryName:
        print("ignores.append(\"{0}\")".format(primaryName))

In [ ]:
vals="""
[Zhane                          0.8                           Zhana] 	 --> (Billboard) 851149b6e3d5945a70fdec72f14183e8 / 0000600590 (AllMusic)
"""



lines=vals.split("\n")
lines2=[x.split("({0}) ".format(chartType))[1] for x in lines if len(x) > 0]
lines3={x.split(" / ")[0]: x[x.find("(")+1:-1] for x in lines2}


for k,v in lines3.items():
    #print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', \'{2}\')".format(k,v,))
    print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', None)".format(k,v))
#mdbmaps[chartType].addArtistDataByID("e309c2e3fc905eae304b71e10e82eb99", "AllMusic", '0000803469')
print("mdbmaps[chartType].save()")

In [ ]:
lines=vals.split("\n")
lines2=[x.split("(Top40) ")[1] for x in lines if len(x) > 0]
vlines3={x.split(" / ")[0]: x[x.find("(")+1:-1] for x in lines2}

In [ ]:
for line in nvals.split("\n"):
    if len(line) > 0:
        try:
            line2=line.split("(Top40) ")[1]
        except:
            continue
        key = line2.split(" / ")[0]
        val = line2[line2.find("(")+1:-1]
        if vlines3.get(key) is None:
            print(line)

In [ ]:
ignores.append("Mr On Vs The Jungle")
ignores.append("Robbie Gerrard Matthew")
ignores.append("LIV'n'G")
print(len(list(set(ignores))))
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")


In [ ]:
mdbmaps[chartType].addArtistDataByID('851149b6e3d5945a70fdec72f14183e8', 'AllMusic', None)
mdbmaps[chartType].save()

In [ ]:
mdbmaps["Billboard"].getPrimaryKey(artistName="Dave Rowland And Sugar")

In [ ]:
analyzeRenames(dbRenames)

In [ ]:
# Test for overap with manDB

dbRenames = getFile(ifile="relDBRenames2.yaml")
redos = {}
dels  = []
for oldername,bestname in dbRenames.items():
    if oldername in manDB.artistNameDB.keys(): # and bestname not in manDB.artistNameDB.keys():
        #print("Older=[{0}] \t Best=[{1}]".format(oldername,bestname))

        redos[bestname] = oldername
        dels.append(oldername)
        #print('PROBLEM ({0} in master DB): \t{1}  -->  {2}'.format(oldername, oldername, bestname))
        try:
            matchOlderData = mdbmaps[chartType].getArtistDataByName(oldername).getDict()
        except:
            matchOlderData = {}
            
        try:
            matchBestData  = mdbmaps[chartType].getArtistDataByName(bestname).getDict()
        except:
            matchBestData = {}
            
        #print("Older: {0}".format(matchOlderData))
        #print("Best:  {0}".format(matchBestData))
        #print("\n")
    else:
        print("{0}: {1}".format(oldername, bestname))

print("# Renames: {0}".format(len(dbRenames)))
print("# Dels: {0}".format(len(dels)))

In [ ]:
toget

In [ ]:
toget

In [ ]:
manDB = masterArtistNameDB("main", init=False)
saveFile(idata=manDB.dbRenames, ifile="main.yaml")

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()
manDB.save()
manDB = masterArtistNameDB("main", init=False)
saveFile(idata=manDB.dbRenames, ifile="main.yaml")

In [ ]:
for k,v in dbRenames.items():
    kKey = mdbmaps[chartType].getPrimaryKey(artistName=k, artistID=None)
    if kKey is not None:
        mdbmaps[chartType].removeArtistByKey(kKey)
        
    kKey = mdbmaps[chartType].getPrimaryKey(artistName=v, artistID=None)
    if kKey is not None:
        mdbmaps[chartType].removeArtistByKey(kKey)
mdbmaps[chartType].save()

In [ ]:
def analyzeRenames(dbRenames):
    redos = {}
    dels = []
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys() and bestname not in manDB.artistNameDB.keys():
            print(oldername,bestname)

            redos[bestname] = oldername
            dels.append(oldername)
            print('\t',oldername,'\t',bestname)

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))
    for oldername in dels:
        del dbRenames[oldername]

    print("# Redos: {0}".format(len(redos)))
    dbRenames.update(redos)
    print("# Renames: {0}".format(len(dbRenames)))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    print(len(dbRenames))
    dbRenames.update(redos)
    print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    print(len(dbRenames))
    dbRenames.update(redos)
    print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    print(len(dbRenames))
    dbRenames.update(redos)
    print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    print(len(dbRenames))
    dbRenames.update(redos)
    print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    print(len(dbRenames))
    dbRenames.update(redos)
    print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")







    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    print(len(dbRenames))
    dbRenames.update(redos)
    print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")
    print(len(dbRenames))

In [ ]:
renames = {"Benedictines of Mary, Queen of Apostles": "Benedictines Of Mary, Queen Of Apostles", "Jhene Aiko": "Jhené Aiko", "Karlos Rose": "Karlos Rosé"}

In [ ]:
for k,v in dbRenames.items():
    rm = chartManDB.renamed(k)
    if dbManDB.removeArtist(rm) != rm:
        print("Not good")
    print(k,v,'\t\t',chartManDB.renamed())

In [ ]:
saveFile(idata=chartManDB.dbRenames, ifile="chartTmpDB.yaml")

In [ ]:
chartDBRenames = getFile("chartTmpDB.yaml")
chartManDB = masterArtistNameDB("chart", init=True)
chartManDB.forceReload(chartDBRenames)
chartManDB.addRenames(getFile("relDBRenames2.yaml"))
chartManDB.save()
chartManDB = masterArtistNameDB("chart", init=False)

In [ ]:
chartManDB = masterArtistNameDB("chart", init=False)

In [ ]:
manDB = masterArtistNameDB("master", init=False)
dbManDB = masterArtistNameDB("db", init=False)
chartManDB = masterArtistNameDB("chart", init=False)
artistNames = manDB.artistNameDB.keys()
for artistName in artistNames:
    if dbManDB.renamed(artistName) != artistName:
        print(artistName,"\t<---- DB Renaming a Master")
    if chartManDB.renamed(artistName) != artistName:
        print(artistName,"\t<---- Chart Renaming a Master")
#manDB

In [ ]:
ignores = getFile("chartIgnores.yaml")
ignores.append("Rev. F.C. Barnes & Rev. Janice Brown")
ignores.append("Rev. J.Cleveland/Southern California Community Choir")
ignores.append("V.I.P. Music & Arts Seminar Mass Choir")
ignores.append("Fred Hammond & Radical For Christ")
ignores.append("Bishop T.D. Jakes And The Potter's House Mass Choir")
ignores.append("Youth For Christ")
ignores.append("Bishop Noel Jones Presents The City Of Refuge Sanctuary Choir")
ignores.append("pureNRG")
ignores.append("Passion Worship Band")
ignores.append("Thomas Miles aka Nephew Tommy")
ignores.append("Queen Latifah, D. Parton, K. Palmer, J. Jordan, A. Grovey, D. Moore & A. Karl")
ignores.append("Whitney Houston & R. Kelly")
ignores.append("Brian Brushwood Justin Robert Young")
ignores.append("Deitrick Haddon's LXW (League Of Xtraordinary Worshippers)")
ignores.append("Swoope")
ignores.append("Phil Wickham With Kari Jobe, Shane & Shane & Jeremy Riddle")
ignores.append("Braiden Sunshine")
ignores.append("Lindy Conant & The Circuit Riders")
ignores.append("Myron Butler & Levi")
ignores.append("My Dad Wrote A Porno")
ignores.append("Deitrick Haddon & Hill City Worship Camp")
ignores.append("Anita Wilson Accompanied By The Company")
ignores.append("Vincent Bohanan And The Sound Of Victory Fellowship Choir")
ignores.append("Gallant x Tablo x Eric Nam")
ignores.append("Snoop Dogg & Various Artists")
ignores.append("Karen Gibson & The Kingdom Choir")
ignores.append("Fresh Start Worship")
ignores.append("Erica Campbell X Warryn Campbell")
print(len(list(set(ignores))))
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")

# Investigate Matches

In [ ]:
ignores = getFile("chartIgnores.yaml")
mbDF = mdbmc.getMasterDF("Billboard")
mbDF = mbDF[~mbDF["Artist"].isin(ignores)]
mbDF.shape

In [ ]:
dbName = "AllMusic"
for idx,row in mbDF[mbDF.DBMatches == 0].iterrows():
    albums = singleArtistAlbumData[chartType][row["Artist"]]
    print("add{0}(mdbmaps, {1}, {2}, {3})   ### {4}".format("AllMusic", "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "None", row["Artist"]))
    print("add{0}(mdbmaps, {1}, {2}, {3})   ### {4}".format("Discogs", "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "None", row["Artist"]))
    print("### {0}\n".format("  ,  ".join(albums)))

In [ ]:
addAllMusic(mdbmaps, "Billboard", "50c6c35f3a878e8311978036c79baf9c", 'mn0000859509')   ### Quicksand
### Interiors

addAllMusic(mdbmaps, "Billboard", "cc4f66ab0c5212c5e228b56645401b45", 'mn0001040351')   ### k.flay
### Every Where Is Some Where

addAllMusic(mdbmaps, "Billboard", "049409bf24127a07844a2807e6516eca", 'mn0003358672')   ### GENTRI
### Finding Christmas

addAllMusic(mdbmaps, "Billboard", "2f4214212c2d6496cfff6147f18e24f0", 'mn0001913938')   ### Chuck Murphy
### Peace Be With You

addAllMusic(mdbmaps, "Billboard", "9406afcb46bcfe434d25670411913ae8", 'mn0003463938')   ### ANOHNI
### HOPELESSNESS

addAllMusic(mdbmaps, "Billboard", "75beb56f8a39c26807d21c698a33264f", 'mn0003598738')   ### Jess And Gabriel
### Under The Covers (EP)

addAllMusic(mdbmaps, "Billboard", "86bc7f3d9b6ea557365ff18ff8b1709a", 'mn0003526563')   ### MO3
### 4 Indictments

addAllMusic(mdbmaps, "Billboard", "7885075dda1d651afbc24d02ff966088", "mn0000990384")   ### Oh Sees
### Smote Reverser

addAllMusic(mdbmaps, "Billboard", "3c54b2bd5cd420fc4ee4ffe258c41e81", "mn0000683342")   ### YOB
### Our Raw Heart

addAllMusic(mdbmaps, "Billboard", "28d561b2325276dbd365b81c37ba4e73", "mn0003595642")   ### Muscadine Bloodline
### Movin' On (EP)

addAllMusic(mdbmaps, "Billboard", "67f687557028fd1aec6c79b365574961", "mn0003739239")   ### Tammy Rivera
### Fate (EP)

addAllMusic(mdbmaps, "Billboard", "e3a9c105fdead1e8eb66eb2ccc8aa120", "mn0001569643")   ### Turnstile
### Time & Space

addAllMusic(mdbmaps, "Billboard", "ba8ddeb4e12ec15d04c28c7a1b800ccf", "mn0003497324")   ### Saddleback Worship
### F I R S T (EP)

addAllMusic(mdbmaps, "Billboard", "741cd131a52950a8b05f7d7cf2d59211", "mn0002515996")   ### Tennis
### Young & Old

addDiscogs(mdbmaps, "Billboard", "facb2fb04f31db2f9fbeb65e39b79f39", "253268")   ### H20
### Use Your Voice

addAllMusic(mdbmaps, "Billboard", "48198ad7a104d82cecd9a93d6f524413", "mn0002674457")   ### FIDLAR
### Too

addAllMusic(mdbmaps, "Billboard", "3fc86bdffcc9447fbb57d6a624af77b8", "mn0003011607")   ### S O H N
### Tremors

addAllMusic(mdbmaps, "Billboard", "433db998514898863897cc2f587ae223", "mn0003434594")   ### Louisa Wendorff
### Arrow (EP)

addAllMusic(mdbmaps, "Billboard", "306e1807db124e4b6b3793e24a8956d6", "mn0000156053")   ### U.S. Bonds
### Quarter To Three

addAllMusic(mdbmaps, "Billboard", "bf2ad6b61124e7209741c6967ee8fabe", "mn0000682096")   ### Hitman Sammy Sam
### Step Daddy

addAllMusic(mdbmaps, "Billboard", "652f0ddefc8da23d94488ff64edc12ce", "mn0000803774")   ### G. Dep
### Child Of The Ghetto

addAllMusic(mdbmaps, "Billboard", "2c72850c58389e5f132fe6f95bb76b1f", "mn0000749222")   ### Paul Oakenfold
### Swordfish: The Album (Soundtrack)

addAllMusic(mdbmaps, "Billboard", "4a1f7e5da0c36d9a865cb59b8430b8f3", "mn0000621869")   ### Bobby Pinson
### Man Like Me

addAllMusic(mdbmaps, "Billboard", "893b555ad03cd9ac30da8fbe4aedf811", "mn0000153630")   ### FRENTE!
### Marvin The Album

mdbmaps["Billboard"].save()

In [ ]:
print(len(ignores))
ignores += ['Phil Collins & Marilyn Martin',
 'Linda Ronstadt (Featuring Aaron Neville)',
 'Anthony Kearns/Ronan Tynan/Finbar Wright',
 'Broken Social Scene Presents: Kevin Drew',
 'Eminem, Dr. Dre & 50 Cent',
 'Jay Farrar & Benjamin Gibbard',
 'Mark Olson & Gary Louris',
 'Ray J & Yung Berg',
 'Kermit Ruffins & Irvin Mayfield',
 'G-Eazy x Bebe Rexha',
 'Sam Beam & Jesca Hoop',
 'Sam Smith & Normani',
 'benny blanco, Halsey & Khalid',
 'Smith And Hay, DJ Whoo Kid & Ranna Royce',
 'CZARFACE / MF Doom',
 'Kanye West & Lil Pump',
 'G-Eazy & Halsey',
 'Cardi B, Bad Bunny & J Balvin',
 'Connor Christian & Southern Gothic',
 'Russell Moore & IIIrd Tyme Out',
 'The Grouch & Eligh',
 'A Great Big World & Christina Aguilera',
 'Idina Menzel Duet With Michael Buble',
 'Snarky Puppy & Metropole Orkest',
 'DJ Snake & AlunaGeorge',
 'Pitbull & Ne-Yo',
 'OCD: Moosh & Twist',
 'Kenny Rogers With Kim Carnes & James Ingram',
 'Mike Reno And Ann Wilson',
 'Joe Cocker And Jennifer Warnes',
 'Carl Anderson & Gloria Loring',
 'Michael Jackson With Siedah Garrett',
 'Art Garfunkel With James Taylor & Paul Simon',
 'Teri DeSario With K.C.',
 'Dionne Warwick & Jeffrey Osborne',
 'Bill Medley & Jennifer Warnes',
 'Vanessa Williams/Brian McKnight',
 'D.J. Magic Mike & M.C. Madness',
 'Rod Stewart With Ronald Isley',
 'The Chipmunks With David Seville',
 'John Lennon With The Plastic Ono Nuclear Band',
 'John Fred And The Playboys',
 'Eric Weissberg & Steve Mandell',
 'Sammy Davis, Jr. w/ The Mike Curb Congregation',
 'John P. Kee And New Life',
 'Usher And Alicia Keys',
 'Nelly, P. Diddy & Murphy Lee',
 'B2K & P. Diddy',
 'Jim Brickman With Kristy Starling',
 'Bishop Eddie L. Long Presents New Birth Total Praise Choir',
 "John Mellencamp With Me'Shell Ndegeocello",
 'Mariah Carey & Boyz II Men',
 'John McDermott/Anthony Kearns/Ronan Tynan',
 'Puff Daddy (Featuring Mase)']
print(len(list(set(ignores))))
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")

In [ ]:
mbDF[mbDF.DBMatches == 0].index

In [ ]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000000, maxAlbums=10000000, minAlbums=0, useKnown=True, ignores=["chartIgnores.p"])
len(toMatch["Billboard"])

In [ ]:
mbDF[mbDF.DBMatches == 0].index

In [ ]:
dbName = "AllMusic"
for idx,row in mbDF[(mbDF["DBMatches"] == 0) & (mbDF["Albums"] > 1)].iterrows():
    print("add{0}(mdbmaps, {1}, {2}, {3})   ### {4}".format(dbName, "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "None", row["Artist"]))
    print("### {0}\n".format("  ,  ".join(["[{0}]".format(x) for x in matchData[idx]["ArtistAlbums"]])))
    

In [ ]:


addAllMusic(mdbmaps, "Billboard", "dc135230d14fb0e9bdc1460769a8b1a1", "mn0000476599")   ### Collie Buddz
### [Collie Buddz]  ,  [Good Life]  ,  [Hybrid]

addAllMusic(mdbmaps, "Billboard", "1152bdd93f6e662ff5690599967b1c35", "mn0001259928")   ### 2PM
### [Winter Games]  ,  [Midaretemina]  ,  [Guilty Love]

addAllMusic(mdbmaps, "Billboard", "3108afa4f2b3c4750753ed4151af005b", "mn0003015318")   ### Benedictines Of Mary, Queen Of Apostles
### [Angels And Saints At Ephesus]  ,  [Lent At Ephesus]  ,  [Easter At Ephesus]

addAllMusic(mdbmaps, "Billboard", "7cd279df51f125cc2541381ea7ec9b7a", "mn0003442077")   ### Nephew Tommy
### [Cheaters: Nephew Tommy Prank Calls Volume 6]  ,  [Celebrity Prank Phone Calls]  ,  [Won't He Do It!: Prank Phone Calls]

addAllMusic(mdbmaps, "Billboard", "f0789f3261234ae074629c6df10f8d45", "mn0000209724")   ### Charlie Cruz
### [Tu Me Confundes]  ,  [Dejame Cambiarte La Vida]  ,  [Quiero Saber de Ti]

addAllMusic(mdbmaps, "Billboard", "aa5f602d8417d9e174d0c8f1a35656a5", "mn0000029884")   ### Paul McCartney/Wings
### [My Love]  ,  [Silly Love Songs]  ,  [Let 'Em In]

addAllMusic(mdbmaps, "Billboard", "47656abc5cb43b94faf471847d41efdf", "mn0002333502")   ### G-Dragon
### [One Of A Kind (EP)]  ,  [Coup D'etat, Pt. 1 (EP)]  ,  [KWON JI YOUNG (EP)]

addAllMusic(mdbmaps, "Billboard", "20fd3e1bc52386bb2acca90205ecf3d9", "mn0000946922")   ### Héctor Acosta "El Torito"
### [Sin Perdon]  ,  [Tu Veneno]  ,  [Amorcito Enfermito]

addAllMusic(mdbmaps, "Billboard", "db16971ae425c378b795c0450265cbdc", "mn0003052164")   ### CHVRCHES
### [Bones Of What You Believe]  ,  [Every Open Eye]  ,  [Love Is Dead]

addAllMusic(mdbmaps, "Billboard", "030e4e18e001df04d1297fbc65e6a2e5", "mn0000294377")   ### Linda Clifford
### [If My Friends Could See Me Now]  ,  [SHOOT YOUR BEST SHOT/IT DON'T HURT NO MORE]  ,  [DON'T COME CRYING TO ME/LET IT RIDE]

addAllMusic(mdbmaps, "Billboard", "1f2dc3cd16e69eb70c6d7619684bba27", "mn0000670057")   ### Dan Hartman
### [Instant Replay (all cuts)]  ,  [VERTIGO/RELIGHT MY FIRE/FREE RIDE]  ,  [We Are The Young]

addAllMusic(mdbmaps, "Billboard", "c4879d2fd7aa56353a74b7ed1c2984c1", "mn0001411997")   ### Dave Rowland And Sugar
### [The Door Is Always Open]  ,  [Tear Time]  ,  [Golden Tears]

addAllMusic(mdbmaps, "Billboard", "5987eb8c3103d8999da70c7dfceced60", "mn0000598526")   ### Armik
### [Barcelona]  ,  [Enamor]  ,  [Pacifica]

addAllMusic(mdbmaps, "Billboard", "8cf125b4398ec24a60757a1ae0dbde6a", "mn0000695963")   ### Yuri
### [QUE TE PASA]  ,  [HOMBRES AL BORDE DE UN ATAQUE DE CELOS]  ,  [Detras De Mi Ventana]

ignores.append("J Balvin & Bad Bunny")
### [Que Pretendes]  ,  [La Cancion]  ,  [Oasis]

addAllMusic(mdbmaps, "Billboard", "0bf355ba1a6225a9880b0256d189b2a3", "mn0000855780")   ### Najee
### [Sweet Summer Nights]  ,  [Perfect Nites]  ,  [Fly With The Wind]

addAllMusic(mdbmaps, "Billboard", "d9093d2d5384cd8c905366d79a2e28b4", "mn0000214245")   ### Los Yonic's
### [Los Yonics]  ,  [PETALOS Y ESPINAS]  ,  [Siempre Te Amare]

addAllMusic(mdbmaps, "Billboard", "10012ea952796b2b799eae2301a5cab6", "mn0000169578")   ### Franky Ruiz
### [Solista pero no solo]  ,  [Voy pa' encima]  ,  [Mas Grande Que Nunca]

addAllMusic(mdbmaps, "Billboard", "c85aa991e15fbab8b0770e07d5f04692", "mn0000169578")   ### Frankie Ruiz
### [HISTORIA MUSICAL DE FRANKIE RUIZ]  ,  [Mirandote]  ,  [Ironia]

addAllMusic(mdbmaps, "Billboard", "78bd920d92646eaea2e98472393634db", "mn0000663790")   ### Gloria Estefan & Miami Sound Machine
### [Can't Stay Away From You]  ,  [Anything For You]  ,  [1-2-3]

addAllMusic(mdbmaps, "Billboard", "3bbeda7f2ded980f784a51329ae7c96b", "mn0000261461")   ### Jose Luis Rodriguez
### [Y TU TAMBIEN LLORARAS]  ,  [SENOR CORAZON]  ,  [BAILA MI RUMBA]

addAllMusic(mdbmaps, "Billboard", "99ccbacd8a76cd01ae7f124c403d252b", "mn0000066318")   ### Paul Taylor
### [Burnin']  ,  [Push To Start]  ,  [Supernova]

ignores.append("Bronco/Los Bukis")
### [Cronica De Dos Grandes]  ,  [Cronica De Dos Grandes: Recuerdos Con Amor]  ,  [Cronica De Dos Grandes: Los Inicios De Nuestra Historia]

addAllMusic(mdbmaps, "Billboard", "f22cea63cdc368979647cba543b07584", "mn0000074784")   ### bwb
### [Shake Your Body (Down To The Ground)]  ,  [bwb]  ,  [Triple Dare]

addAllMusic(mdbmaps, "Billboard", "41a0eaa369b77a12da16fa7cc53c8e6e", "mn0000251390")   ### VaShawn Mitchell
### [Nobody Greater]  ,  [Secret Place: Live In South Africa]  ,  [Joy]

addAllMusic(mdbmaps, "Billboard", "81e577a3bdf7fa984e583aeb7de6de9d", "mn0000054266")   ### Beto y Sus Canarios
### [Esta Llorando Mi Corazon]  ,  [Ardientes]  ,  [No Puedo Olvidarte]

In [ ]:
ignores = list(set(ignores))
saveFile(idata=ignores, ifile="chartIgnores.yaml")
mdbmaps[chartType].save()

In [ ]:
matchData = {mdbmaps[dbName].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in singleArtistAlbumData[chartType].items()}

In [ ]:
def matchArtistWithMixAlbums(maindb, artistName, artistID, artistAlbums, dbs, kwargs):
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    #mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    mcs = mdbMatcher.findPotentialArtistAlbumMatchesByDBList(dbs)
    return mcs



for xxx in range(10):
    nAdds = 0
    for i,(idx,row) in enumerate(mbDF.iterrows()):
        if i <= iStart:
            continue
        print(xxx,i,iStart,idx)

        if nAdds >= 25:
            mdbmaps[chartType].save()
            iStart = i
            nAdds = 0
            break

        dbIDs = row.to_dict()
        chartArtistName = dbIDs["Artist"]

        try:
            chartArtistAlbums = matchData[idx]["ArtistAlbums"]
        except:
            print("Somehow could not find albums for {0},{1}".format(idx,chartArtistName))

        dbArtistAlbums = {}
        dbsToCheck = []
        for db,dbID in dbIDs.items():
            if db in ["Artist", "Albums", "DBMatches", "DatPiff", "MetalStorm"]:
                continue
            if dbID is not None:
                dbArtistAlbums[db] = maindb.getArtistDBAlbumsFromID(db, dbID)
            else:
                dbsToCheck.append(db)
                
        print("\t",chartArtistName,'\t',dbsToCheck)

        fullArtistAlbums = chartArtistAlbums + getFlatList(dbArtistAlbums.values())
        nAlbs = len(fullArtistAlbums)
        thresholds = getThresholds(None)
        thres = max([x for x in thresholds.keys() if len(chartArtistAlbums) > x], default=None)
        if thres is None:
            nAdds += 1
            continue

        mcs = matchArtistWithMixAlbums(maindb, chartArtistName, idx, fullArtistAlbums, dbsToCheck, thresholds[thres])

        first = True
        for db,mc in mcs.items():
            dbID = mc.matchID
            if dbID is not None:
                if first:
                    print("{0}   {1}/{2}".format(chartArtistName,i,mbDF.shape[0]))
                    first = False
                print("\t\t{0} / {1}  [{2}]".format(db, dbID, maindb.getArtistDBNameFromID(db, dbID)))
                mdbmaps[chartType].addArtistDataByKey(idx, db, dbID)
                nAdds += 1

        end = True
        
    print("\n\n",xxx,'\n\n\n')

In [ ]:
iStart = 2453

In [ ]:
toMatch = getDataToMatch("Billboard", mdbmaps, "single", maxValues=`1002, maxAlbums=100010, sort=True)
matchItAll(mdbmaps, toMatch, cutoff=0.99)

In [ ]:
chartTypes = ["Top40", "Billboard"]
artistAlbumsData = {}
for chartType in chartTypes:
    savename        = "current{0}ArtistAlbumData.p".format(chartType)
    artistAlbumsData[chartType] = getFile(savename)

In [ ]:
from multiArtist import multiartist
multiDelimArtists=open("../multiartist/multiDelimArtists.dat").readlines()
multiDelimArtists = [x.replace("\n", "") for x in multiDelimArtists]
multiDelimArtists[:3]

mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
mularts.setKnownMultiDelimArtists(multiDelimArtists)

In [ ]:
singleArtistAlbumData = {"Top40": {}, "Billboard": {}}
manyArtistAlbumData   = {"Top40": {}, "Billboard": {}}

In [ ]:
def createArtistAlbumData(artistAlbumData):
    from multiArtist import multiartist
    multiDelimArtists=open("../multiartist/multiDelimArtists.dat").readlines()
    multiDelimArtists = [x.replace("\n", "") for x in multiDelimArtists]
    multiDelimArtists[:3]

    mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
    mularts.setKnownMultiDelimArtists(multiDelimArtists)    
    
    
    print("There are {0: <6} unique artist entries in the artist albums data".format(len(artistAlbumData)))
    singleArtistAlbumData     = {artistName: artistAlbums for artistName,artistAlbums in artistAlbumData.items() if len(mularts.getArtistNames(artistName)) == 1}
    manyArtistAlbumData       = {}

    print("There are {0: <6} single artist entries in the artist albums data".format(len(singleArtistAlbumData)))


    manyArtists       = {artist: mularts.getArtistNames(artist) for artist in artistAlbumData.keys() if singleArtistAlbumData.get(artist) is None}

    print("There are {0: <6} multi artist entries in the artist albums data".format(len(manyArtists)))

    for manyartist,artists in manyArtists.items():
        manyArtistAlbums = artistAlbumData[manyartist]
        for artistName in artists:            
            if singleArtistAlbumData.get(artistName) is not None:
                singleArtistAlbumData[artistName] += manyArtistAlbums
            else:
                if manyArtistAlbumData.get(artistName) is not None:
                    manyArtistAlbumData[artistName] += manyArtistAlbums
                else:
                    manyArtistAlbumData[artistName] = manyArtistAlbums
                    
    print("There are {0: <6} solo artist entries in the artist albums data".format(len(singleArtistAlbumData)))
    print("There are {0: <6} many artist entries in the artist albums data".format(len(manyArtistAlbumData)))
                    
                    
    return singleArtistAlbumData, manyArtistAlbumData

In [ ]:
for i in range(4):
    toMatch = getDataToMatch("Top40", mdbmaps, "many", maxValues=1002, maxAlbums=100010, sort=True)
    matchItAll(mdbmaps, toMatch, cutoff=0.99)

In [ ]:
toMatch = getDataToReMatch("Billboard", mdbmaps, "single")
matchItAll(mdbmaps, toMatch, cutoff=0.90)

In [ ]:
toMatch["Top40"]

In [ ]:
artistAlbumsData["Top40"]

for i in range(1):
    #for db in ["AllMusic", "MusicBrainz", "LastFM", "RockCorner", "CDandLP", "RateYourMusic", "MusicStack"]:
    for db in ["Music"]: #, "MusicBrainz", "LastFM", "RockCorner", "CDandLP", "RateYourMusic", "MusicStack"]:
        #toMatch   = mdbmc.getDataToMatch(db, maxValues=500, maxAlbums=50000)
        
        if len(toMatch) == 0:
            continue
        

        num_processes = 3
        func = matchDBArtistWithAlbums
        pfunc = partial(matchDBArtistWithAlbums, cutoff=0.95) # Giving some arguments for kwargs
        #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
        dbName = list(toMatch.keys())[0]
        argument_list = toMatch[dbName]
        if len(argument_list) == 0:
            break
        print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
        result_list = multiProc(func=pfunc, argument_list=argument_list,
                                               num_processes=num_processes)


        break
        saveMapData(mdbmaps[dbName], result_list)
        sleep(10)

        #mdbmc.matchMutualMaps()

In [ ]:
newDB = "LastFM"
mydf = mdbmap.getDF().T
mdbMatcher = matchDBArtist(maindb)

results = {}
saves   = 0
for i,(idx,row) in enumerate(mydf.iterrows()):
    if i <= 4152:
        continue
    myArtistName      = idx[0]
    myArtistID        = idx[1]
    

    dbMatchIDs    = row.to_dict()
    dbMatchNames  = {}
    dbMatchAlbums = {}
    
    allAlbums = []

    toMatch = []
    for dbName,dbMatchID in dbMatchIDs.items():
        if dbName in ["DatPiff", "MetalStorm"]:
            continue
        if dbMatchID is None:
            dbMatchNames[dbName]  = None
            dbMatchAlbums[dbName] = []
            toMatch.append(dbName)
            continue
        dbMatchNames[dbName] = maindb.getArtistDBNameFromID(dbName, dbMatchID)
        if dbMatchNames[dbName] is None:
            print(idx,'\t',dbName)
            dbMatchAlbums[dbName] = []
        else:
            dbMatchAlbums[dbName] = maindb.getArtistDBAlbumsFromID(dbName, dbMatchID)
            allAlbums += dbMatchAlbums[dbName]
            
    allAlbums = list(set(allAlbums))
    print(idx,'\t',len(allAlbums))

    
    for dbName in toMatch:
        mdbMatcher = matchDBArtist(maindb)
        mdbMatcher.setArtistInfo(myArtistName, myArtistID, allAlbums)
        #mdbMatcher.findPotentialArtistNameMatches()
        mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB(dbName)
        if mc.matchID is not None:
            if results.get(idx) is None:
                results[idx] = {}
            results[idx][dbName] = mc.matchID
            print('\t\t',i,'\t',idx,' --> \t',dbName,'\t',mc.matchID)
            saves += 1

    if saves > 80:
        break

    continue
    
    
    # New DB
    if row[newDB] is None:
        allMusicDataNewDBID    = allMusicData.getDict()[newDB]
        if allMusicDataNewDBID is not None:
            newDBAlbums = maindb.dbdatamap[newDB].getArtistAlbums(allMusicDataNewDBID, flatten=True)
            mdbMatcher.setArtistInfo(allMusicName, allMusicID, newDBAlbums)
            mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB(newDB)
            matchID = mc.matchID
            if matchID is not None:
                print("Adding From AllMusic")
                #mdbmap.addArtistData(myArtistName, myArtistID, newDB, matchID)
                continue
                
                
        musicBrainzDataNewDBID = musicBrainzData.getDict()[newDB]
        if musicBrainzDataNewDBID is not None:
            newDBAlbums = maindb.dbdatamap[newDB].getArtistAlbums(musicBrainzDataNewDBID, flatten=True)
            mdbMatcher.setArtistInfo(musicBrainzName, musicBrainzID, newDBAlbums)
            mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB(newDB)
            matchID = mc.matchID
            if matchID is not None:
                print("Adding From MusicBrainz")
                #mdbmap.addArtistData(myArtistName, myArtistID, newDB, matchID)
                continue
    
        print("Not Adding Anything")
        


In [ ]:


from chartAnalysisClass import chartAnalysisClass
for chartType in chartTypes:
    cac = chartAnalysisClass(chartType, init=True, full=False)
    cac.setFull(mdb, mcm)
    cac.setChartArtists(split=split)
    cac.findArtistsMatchedStatus()
    cacs[chartType] = cac
    print("\n"*2)

# Match Artists

In [ ]:
mdbChartType = next(iter(cacs.keys()))
cacs[mdbChartType].loadPreviouslyMatchedChartArtists(init=True, split=split)

# Find Matches

In [ ]:
if True:
    mdbChartType = next(iter(cacs.keys()))
    mdb = cacs[mdbChartType].mdb
    mcm = cacs[mdbChartType].mcm

    for chartType in cacs.keys():
        cac = cacs[chartType]
        #test = matchIt(cac, -1)
        cac = matchIt(cac, 1, maxValues=3000)
        cac.saveMatchedChartArtists()
        cacs[chartType] = cac
        break

In [ ]:
Series({}).nunique() > 0

In [ ]:
if False:
    mdbChartType = next(iter(cacs.keys()))
    mdb = cacs[mdbChartType].mdb
    mcm = cacs[mdbChartType].mcm

    for chartType in cacs.keys():
        continue
        cac = cacs[chartType]

        ################################################################################
        ################ Single    
        ################################################################################
        #cac = matchIt(cac, 0)
        #cac = matchIt(cac, 2, maxValues=None)  ## 5.5 minutes ~350 seconds / 1000 for Pure,  2.5 minutes ~200 seconds / 500 for VeryPure, 17.3 minutes w/ 2500 for VeryPure
        #cac = matchIt(cac, 1, maxValues=None)


        ################################################################################
        ################ Many
        ################################################################################
        #cac = matchIt(cac, 5)
        #cac = matchIt(cac, 7, maxValues=500)  ## 5.5 minutes ~350 seconds / 1000 for Pure,  2.5 minutes ~200 seconds / 500 for VeryPure, 17.3 minutes w/ 2500 for VeryPure
        cac = matchIt(cac, 6, maxValues=None)
        #cac = matchIt(cac, -1)
        cac.saveMatchedChartArtists()
        cacs[chartType] = cac

        break

# Show Missing

In [ ]:
if False:
    for chartType in cacs.keys():
        cac = cacs[chartType]
        unMatchedSingleArtists      = cac.unMatchedSingleArtists
        unMatchedSingleArtistAlbums = {k: v for k,v in cac.singleArtistAlbums.items() if k in unMatchedSingleArtists}
        break

In [ ]:
if False:
    for artist,albums in unMatchedSingleArtistAlbums.items():
        print("{0: <30}{1}".format(artist, albums))

In [ ]:
if False:
    for artist in cac.unMatchedSingleManyArtists:
        print("{0: <30}{1}".format(artist, cac.reverseManySingleArtists.get(artist)))

In [ ]:
if False:
    for chartType in cacs.keys():
        cac = cacs[chartType]
        pairs = cac.getArtistAlbumSearchPairs("Many")
        unMatchedSingleManyArtists = cac.rev
        #return [self.unMatchedSingleManyArtists, self.manySingleArtistAlbums]
        break

# Music Data Match

In [ ]:
#mcm.setChartArtistData("Nelly", [])
#mcm.matchChartArtistByName()

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
artistRenames = getFile("masterRename.yaml")
from musicData import musicData
from musicDBData import musicDBData
from musicArtistData import musicArtistData
md = musicData(init=True)


### Set MusicDB Data
mdbChartType = next(iter(cacs.keys()))
md.setMusicDBData(cacs[mdbChartType].mdb, debug=True)

### Set ChartDB Data
for chartType,chartData in cacs.items():
    md.setChartDBData(chartType, chartData.matchedChartResults, debug=True)
    
### Set Rename Data    
md.setRenameData(artistRenames, debug=True)

### Save
md.save()


### Master DF
md.createMasterDF()

_ = clock("Last Run")

## Matched DataFrames

In [ ]:
masterdf = getFile("/Users/tgadfort/opt/anaconda3/envs/py37/musicdb/masterMusicData.p")
musicdf  = getFile("/Users/tgadfort/opt/anaconda3/envs/py37/musicdb/musicData.p")

In [ ]:
output = {}
for i,artistName in enumerate(musicdf.keys()):
    output[artistName] = {}

    tmp = musicdf[artistName].dfSummary()
    knownMusic = "Music" in tmp.columns
    for col in set(["Music", "Top40", "Billboard"]).difference(set(tmp.columns)):
        tmp[col] = None
    uniques = {col: [x for x in tmp.T[col].unique() if x is not None] for col in tmp.T}
    uniques = { (col): (keys[0] if len(keys) > 0 else None) for col, keys in uniques.items() }
    
    consensus = uniques
    output[artistName]["Matches"] = len([k for k,v in uniques.items() if v is not None])
    output[artistName]["Music"]   = knownMusic    
    output[artistName]["DBID"]    = consensus
    
outDF = DataFrame(output).T

In [ ]:
saveFile(ifile="ChartMusicDBIDDF.p", idata=outDF)

## DB DataFrames

In [ ]:
dbad = dbArtistData(maindb.dbdata, "AllMusic")
dbad.setMasterRenameFile()
dbad.createArtistNames()
dbad.createArtistAlbums(sumit=False)
dbad.merge()
df = dbad.get()
#dbad.merge()

dfd = df.to_dict()
idxs    = dfd.keys()
artists = dfd['Name'].values()
albums  = dfd['nAlbums'].values()
data = dict(zip(artists, albums))
saveFile(idata=data, ifile="currentAllMusicArtistAlbumData.p")
saveFile(idata={}, ifile="currentAllMusicFullChartArtistAlbumData.p")

In [ ]:
df[df["Name"] == "Bliss"]

In [ ]:
df["Name"].value_counts()

In [ ]:
df["Name"].nunique()

In [ ]:
dbA = dbAlbums(mdb, maindb.dbdata)
dbA.setDBName("AllMusic")
dbA.setMasterRenameFile("masterRename.yaml")
dbA.createArtistMatchesDF()
dbA.createArtistAlbumsDF()
dbA.createArtistIDDF()
dbA.createArtistURLDF()
dbA.join()
rankedDF = dbA.getDF()
#rankedDF = dbA.getArtistsByMostAlbums("AllMusic")

_, _ = clock("Last Run")

In [ ]:
dutils = lastfmUtils()
dutils.getArtistID('Billie Holiday')

In [ ]:
mdb.getArtistData("Johann Sebastian Bach")

In [ ]:
rankedDF

In [ ]:
rankedDF[(rankedDF["Match"].isna()) & (rankedDF["nAlbums"] > 200)]

In [ ]:
rankedDF[~((rankedDF["Match"].isna()) & (rankedDF["nAlbums"] > 100) & (rankedDF["Name"].str.contains(" feat ")))]

In [ ]:
for artistName in mdb.get().keys():
    mdb.initArtistDB(artistName, "LastFM")
mdb.save()

In [ ]:
cut = 0.1
dbmap = {}
DBNames = rankedDF[(rankedDF.Same == False) & (rankedDF.Ratio > cut)]["Name"].values
MyNames = rankedDF[(rankedDF.Same == False) & (rankedDF.Ratio > cut)]["Match"].values
for dbName, myName in dict(zip(DBNames, MyNames)).items():
    if dbName != myName:
        dbmap[dbName] = myName

In [ ]:
for dbName,myName in dbmap.items():
    print("{0}: {1}".format(dbName,myName))
    
print("")
for dbName,myName in dbmap.items():
    print("mdb.add(\"{0}\", \"{1}\", \"\")".format(myName, "AllMusic"))
    
print("")
for dbName,myName in dbmap.items():
    print("mdb.initArtistDB(\"{0}\", \"{1}\")".format(myName, "AllMusic"))
    
#mn0000891063

In [ ]:
mdb.add("The Misfits", "AllMusic", "0000891063")
mdb.initArtistDB("Glimmer", "AllMusic")
mdb.rmArtist("Santana Brothers")
mdb.save()

In [ ]:
mdb.getArtistData("Billie Holiday")

In [ ]:
mdb.rmArtist("The Fifth Dimension")
mdb.save()

In [ ]:
mdb.add("Mancake", "Discogs", "1444791")
mdb.save()

In [ ]:
ignores = {"Los Tiranos del Norte": "Los Tigres Del Norte"}
Witch Mountain: Big Mountain

In [ ]:
mdb.rmArtist("The Marquees")
mdb.save()

In [ ]:
mdb.add("Ernie Hines", "AllMusic", "0000205373")
mdb.add("Big Mountain", "AllMusic", "0000070704")
mdb.save()

In [ ]:
dbDF[dbDF.index.isin(['0000817315', '0000726082', '0002768705', '0000516929', '0000422575'])].join(dbA.mDF.head())

In [ ]:
dbA.mDF.head().index

In [ ]:
chartType="Top40"
data = getFile("current{0}ArtistAlbumData.p".format(chartType))

In [ ]:
data

In [ ]:
for idx,row in rankedDF[rankedDF["Same"] == False].iterro

In [ ]:
print("Svalue = {0}".format(Svalue))
if Svalue < 100:
    Qvalue = rankedDF['nAlbums'].min()
    print("\tS < 100")
    print("\tQvalue = {0}".format(Qvalue))
else:
    Qvalue = rankedDF['nAlbums'].quantile(0.9)
    print("\tS >= 100")
    print("\tQvalue = {0}".format(Qvalue))
print("Rdf = {0}".format(rankedDF.shape[0]))
toget = rankedDF[rankedDF['nAlbums'] >= Qvalue]
print("Rdf = {0}".format(toget.shape[0]))

In [ ]:
toget[["Name", "Match", "nAlbums"]]

In [ ]:
{ (some_key if condition else default_key):(something_if_true if condition
          else something_if_false) for key, value in dict_.items() }

In [ ]:

        artistDBData = {}
        artistData   = self.dbdata
        artistDBData = {dbName: artistData[dbName].getDict() for dbName in artistData.keys()}
        
        summaryDF = DataFrame(artistDBData)
        return summaryDF

In [ ]:
tmp = df['"Weird Al" Yankovic'].dfSummary()
tmp["Top40"] = None
tmp

In [ ]:
df["UB40"].dfSummary()

In [ ]:
mdd = musicData(init=False)

In [ ]:
help(mdd)

In [ ]:
mdd.createMasterDF

In [ ]:
status = []
for idx,row in dfSummary.iterrows():
    status.append(Series([x for x in row.values if x is not None]).nunique())
dfSummary["Status"] = status
dfSummary

In [ ]:
from pandas import concat
tmp = {artistName: md.artists[artistName].dfSummary() for artistName in md.artists.keys()}
alldf = concat(tmp.values(), keys=tmp.keys())
alldf["Status"] = alldf.apply(func=lambda x: x.nunique(), axis=1)
alldf.index.names = ["Artist", "DB"]

In [ ]:
alldf.head(20)

In [ ]:
artists = set(alldf.index.get_level_values("Artist"))

In [ ]:
alldf[alldf.index.get_level_values("DB") == 'AceBootlegs']

In [ ]:
import numpy as np
from numpy.random import randint
A = np.array([1.1, 1.1, 3.3, 3.3, 5.5, 6.6])
B = np.array([111, 222, 222, 333, 333, 777])
C = randint(10, 99, 6)
df = DataFrame(zip(A, B, C), columns=['A', 'B', 'C'])
df.set_index(['A', 'B'], inplace=True)

In [ ]:
df.index

In [ ]:
result_df = df.loc[(df.index.get_level_values('A') > 1.7) & (df.index.get_level_values('B') < 666)]
result_df

In [ ]:
artistName   = "UB40"
artistDBData = {}
artistData   = md.artists[artistName].dbdata
artistDBData = {dbName: artistData[dbName].getDict() for dbName in artistData.keys()}
DataFrame(artistDBData)

In [ ]:
md.artists

In [ ]:
from pandas import DataFrame, Series, concat
def cleanDF(artistName, df):
    retval = df.reset_index()
    retval["Name"] = artistName
    return retval


tmp = [cleanDF(artistName, artistData.dfSummary()) for artistName, artistData in md.artists.items()]

#tmp = concat(DataFrame({artistName: cleanDF(artistData.dfSummary()) for artistName, artistData in md.artists.items()}), axis=0)

In [ ]:
ctmp = concat(tmp)
ctmp[["Name"] == "UB40"]

In [ ]:
df.index

In [ ]:
#matchIt(1)
#matchIt(4)

In [ ]:
#cac.summary()

In [ ]:
from pandas import Series
inputs = Series({k: len(v) for k,v in {artist: list(set(cac.singleArtistAlbums.get(artist))) for artist in cac.unMatchedSingleArtists}.items()}).sort_values(ascending=False)
for artist in inputs.index:
    if getArtistIgnores(artist):
        albums = cac.singleArtistAlbums.get(artist)
        if albums is None:
            albums = [None, None]
        print("{0: <40}{1: <5}{2}".format(artist,inputs[artist],albums[:2]))

In [ ]:

md.save()

***

# Rename Matched Artists

In [ ]:
results

In [ ]:
matchedChartResultsTop40     = getMatchedChartArtists("Top40")
matchedChartResultsBillboard = getMatchedChartArtists("Billboard")

fullChartArtistAlbumDataTop40     = getFullChartArtistAlbumData("Top40")
fullChartArtistAlbumDataBillboard = getFullChartArtistAlbumData("Billboard")

matchedChartResults = {"Top40": matchedChartResultsTop40, "Billboard": matchedChartResultsBillboard}[chartType]

In [ ]:
matchedRenames   = {}
bestMatchedNames = []
removes          = {}
matchedChartResultsIDs = {}

matchedChartResults = {"Top40": matchedChartResultsTop40, "Billboard": matchedChartResultsBillboard}[chartType]

for artist,artistData in matchedChartResults.items():
    for db,dbmatch in artistData.items():
        if matchedChartResultsIDs.get(db) is None:
            matchedChartResultsIDs[db] = {}        
        if dbmatch is not None:
            if not isinstance(db,str) or not isinstance(dbmatch,(str,int)):
                print("matchedChartResults{0}[\"{1}\"][\"{2}\"] = None".format(chartType, artist, db))
                continue
                #raise ValueError("Error with db {0} or match {1}".format(db,dbmatch))
            if matchedChartResultsIDs[db].get(dbmatch) is None:
                matchedChartResultsIDs[db][dbmatch] = artist
            else:
                prevArtist = matchedChartResultsIDs[db][dbmatch]
                #print("# Multiple Matches For DB/ID [{0}/{1}] --> [{2}  ;  {3}]".format(db,dbmatch,artist,prevArtist))
                
                dbName = mdb.dbdata[db].getArtistNameFromID(dbmatch)


                
                if dbName == artist:
                    print("matchedChartResults{0}[\"{1}\"][\"{2}\"] = None".format(chartType, prevArtist, db))
                    #print("#matchedRenames[\"{0}\"] = \"{1}\"".format(prevArtist, artist))
                    #print("#del matchedChartResults{0}[\"{1}\"]".format(chartType, prevArtist))
                    matchedRenames[prevArtist] = artist
                elif dbName == prevArtist:
                    print("matchedChartResults{0}[\"{1}\"][\"{2}\"] = None".format(chartType, artist, db))
                    #print("#matchedRenames[\"{0}\"] = \"{1}\"".format(artist, prevArtist))
                    #print("#del matchedChartResults{0}[\"{1}\"]".format(chartType, artist))
                else:
                    bestMatchedNames.append([artist, prevArtist])
                #print("")
                continue
                    

                    
                
                if mdb.isKnown(artist) and not mdb.isKnown(prevArtist):
                    print("\tRename {0} --> {1}".format(prevArtist,artist))
                    matchedRenames[prevArtist] = artist
                    removes[prevArtist] = True
                elif not mdb.isKnown(artist) and mdb.isKnown(prevArtist):
                    print("\tRename {0} --> {1}".format(artist,prevArtist))
                    matchedRenames[artist] = prevArtist
                    removes[artist] = True
                elif mdb.isKnown(artist) and mdb.isKnown(prevArtist):
                    print("\tBoth artist names are used...")
                    bestMatchedNames.append([artist, prevArtist])
                elif not mdb.isKnown(artist) and not mdb.isKnown(prevArtist):
                    print("\tMust look this up to see what we need to use.")
                    bestMatchedNames.append([artist, prevArtist])
                    
    if len(matchedRenames) > 25000:
        break

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from matchChartMusic import matchChartMusic
mcm     = getMCM(mdb)
mcm.setMatchedChartArtists(matchedChartResults)
mcm.searchForMutualDBEntries(chartType, maxAdds=10000)

In [ ]:
#matchedRenames["David Latour "] = "David Latour"

for k,v in matchedRenames.items():
    print("{0}: {1}".format(k,v))

In [ ]:
Reba                                             4
MercyMe                                          3
La Adictiva Banda San José De Mesillas           3
Roger                                            3
Jim Johnston                                     3
Shelly West                                      2
Flight Of The Conchords                          2
LTD                                              2
Skrillex                                         2
Jake Hoot                                        2
Glee Cast                                        2
Jordan Smith                                     2
EXO                                              2
Grand Funk                                       2
Big Audio                                        2
Esperanza Spalding                               2
Jermaine Dupri                                   2
Dethklok                                         2
La Arrolladora Banda el Limon de Rene Camacho    2
David Frizzell                                   2
Ricky Doran                                      2
Rival Sons                                       2
Jon Bon Jovi                                     1
Bobby McFerrin                                   1
The O'Kanes                                      1
E.U.                                             1
Michael Damian                                   1
Schuyler, Knobloch And Bickhardt                 1
El DeBarge                                       1
Big Daddy Kane                                   1
Pam Tillis                                       1
Stevie B                                         1
Samuele                                          1
The Rembrandts                                   1
Christopher Williams                             1
The Motels                                       1
Compton's Most The Wanted                        1
UMC's                                            1
M.C. Brain                                       1
Grand Puba                                       1
MC Serch                                         1
Ini Kamoze                                       1
Big Mountain                                     1
Robyn Hitchcock                                  1
BlackPink                                        1
The Tubes                                        1
Blue Magic                                       1
The Pips                                         1
Joey Dee & The Starliters                        1
Ernest                                           1

In [ ]:
for k,v in matchedRenames.items():
    print("{0}: {1}".format(k,v))

# Find Near Matches

In [ ]:
previouslyKnownArtists = getMatchedChartArtists("Top40").keys()
result = {}
for i,artist in enumerate(unMatchedSingleArtists + unMatchedSingleManyArtists):
    print(i,'\t',artist)
    retvals = findNearest(artist, previouslyKnownArtists, num=2, cutoff=0.75)
    print('\t',retvals)
    if len(retvals) == 2:
        nretvals = [x for x in retvals if x != artist]
        num = len(artistAlbumData[artist])
        if num >= 2:
            print("{0: <40}{1: <10}{2}".format(artist,num,nretvals[0]))
            #print("renames[\"{0}\"] = \"\"".format(val))
            result[artist] = {"Num": num, "Possible": nretvals[0]}

In [ ]:
results

In [ ]:
saveMatchedChartArtists(matchedChartResultsTop40, "Top40")

In [ ]:
saveMatchedChartArtists(matchedChartResultsTop40, "Top40")

In [ ]:
for k in removes.keys():
    print("del matchedChartResultsBillboard[\"{0}\"]".format(k))

In [ ]:
saveMatchedChartArtists(matchedChartResultsBillboard, "Billboard")

In [ ]:
matchedRenames

In [ ]:
for i,names in enumerate(bestMatchedNames):    
    bestRenames = {}

    for name in names:
        results = mdb.getNearestArtistNames(name)
        print(i,'/',len(bestMatchedNames),'\t',name,"\t",[x for x in results.values() if x is not None]) #[x for x in results.values() if x is not None])
        bestRenames[name] = [x for x in results.values() if x is not None]
    
    for name in names:
        try:
            print("{0} ---> {1}".format(name,singleArtistAlbums.get(name)))
        except:
            pass
        try:
            print("{0} ---> {1}".format(name,manyArtistAlbums.get(name)))
        except:
            pass

        
    for name,results in bestRenames.items():
        
        print("del matchedChartResultsBillboard[\"{0}\"]".format(name))
        for v in [x for x in results if x is not None]:
            if name == v:
                continue
            print("del matchedChartResultsBillboard[\"{0}\"]".format(v))
            print("{0}: {1}".format(v,name))
    
    print("\n")

In [ ]:
renames = {}
renames["Brian Mckight"] = "Brian McKnight"
renames["Shane Ritchie"] = "Shane Richie"
renames["Jaime Cullum"] = "Jamie Cullum"
renames["Supafly INC"] = "Supafly"
renames["Anitta"] = "MC Anitta"
renames["Bravery"] = "The Bravery"
renames["Pornograffitti"] = "Porno Graffitti"
renames["Tommy Februaryo"] = "Tommy February6"

for k,v in renames.items():
    artistRenames[k] = v
saveArtistRenames(artistRenames)
#saveMatchedChartArtists(matchedChartResultsBillboard, "Billboard")

In [ ]:
for i in range(100):
    print("renames[\"\"] = \"\"")
    print("del matchedChartResultsBillboard[\"\"]")

# Music Renames

In [ ]:
mdb = myMusicDBMap(debug=False)
db  = mdb.get()
renames = getFile("masterRename.yaml")

In [ ]:
md = musicData()
md.setMusicDBData(mdb)
md.setRenameData(renames)

In [ ]:
def loadChartData():
    mularts = getMA()
    artistAlbumData = getArtistAlbumData()
    allArtists      = list(artistAlbumData.keys())
    print("There are {0} artist album entries".format(len(artistAlbumData)))

    fullChartData = getFullChartArtistAlbumData()

    manyArtists   = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) > 1]
    manyArtistAlbums = {artist: {"Artists": mularts.getArtistNames(artist)} for artist in manyArtists}
    print("There are {0} many artist entries".format(len(manyArtists)))

    singleArtists   = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) == 1]
    print("There are {0} single artist entries".format(len(singleArtists)))

    _, _ = clock("Last Run")
    
    return mularts, artistAlbumData, allArtists, fullChartData, manyArtists, manyArtistAlbums, singleArtists


In [ ]:
def loadChartData():
    mularts = getMA()
    artistAlbumData = getArtistAlbumData()
    allArtists      = list(artistAlbumData.keys())
    print("There are {0} artist album entries".format(len(artistAlbumData)))

    fullChartData = getFullChartArtistAlbumData()

    manyArtists   = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) > 1]
    manyArtistAlbums = {artist: {"Artists": mularts.getArtistNames(artist)} for artist in manyArtists}
    print("There are {0} many artist entries".format(len(manyArtists)))

    singleArtists   = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) == 1]
    print("There are {0} single artist entries".format(len(singleArtists)))

    _, _ = clock("Last Run")
    
    return mularts, artistAlbumData, allArtists, fullChartData, manyArtists, manyArtistAlbums, singleArtists




def loadMatchedChartData(singleArtists, manyArtists, artistAlbumData, mularts, init=False):
    matchedChartResults = getMatchedChartArtists(init=init)
    print("="*100)
    print("There are {0} matched chart artists".format(len(matchedChartResults)))
    print("="*50,"Single","="*50)

    singleArtistStatus     = {singleArtist: matchedChartResults.get(singleArtist) is not None for singleArtist in singleArtists}
    singleArtistAlbums     = {}
    for singleArtist in singleArtists:
        singleArtistAlbums[singleArtist] = artistAlbumData[singleArtist]
    matchedSingleArtists   = [singleArtist for singleArtist,isMatched in singleArtistStatus.items() if isMatched is True]
    unMatchedSingleArtists = [singleArtist for singleArtist,isMatched in singleArtistStatus.items() if isMatched is False]
    print("There are {0} single Artists".format(len(singleArtistStatus)))
    print("There are {0} matched Artists".format(len(matchedSingleArtists)))
    print("There are {0} unMatched Artists".format(len(unMatchedSingleArtists)))
    print("="*50,"Many","="*50)

    artistRenames = getArtistRenames()
    manySingleArtists      = {manyArtist: list(mularts.getArtistNames(manyArtist).keys()) for manyArtist in manyArtists}
    manySingleArtistStatus = {}
    manySingleArtistAlbums = {}
    for manyArtist, manySingleArtistValues in manySingleArtists.items():
        for manySingleArtist in manySingleArtistValues:
            if len(manySingleArtist) == 0:
                continue
            renamedManySingleArtist = artistRenames.get(manySingleArtist)
            if renamedManySingleArtist is not None:
                print("\t{0}  <---- From ---- {1}".format(renamedManySingleArtist, manySingleArtist))
                manySingleArtist = renamedManySingleArtist

            if artistAlbumData.get(manySingleArtist) is None:
                artistAlbumData[manySingleArtist] = []
            artistAlbumData[manySingleArtist] += artistAlbumData[manyArtist]
            if manySingleArtistStatus.get(manySingleArtist) is None:
                manySingleArtistStatus[manySingleArtist] = matchedChartResults.get(manySingleArtist) is not None
                manySingleArtistAlbums[manySingleArtist] = {}
            manySingleArtistAlbums[manySingleArtist][manyArtist] = artistAlbumData[manyArtist]

    matchedSingleManyArtists   = [singleArtist for singleArtist,isMatched in manySingleArtistStatus.items() if isMatched is True]
    unMatchedSingleManyArtists = [singleArtist for singleArtist,isMatched in manySingleArtistStatus.items() if isMatched is False]

    print("There are {0} many Artists".format(len(manySingleArtists)))
    print("There are {0} many single Artists".format(len(manySingleArtistStatus)))
    print("There are {0} matched many single Artists".format(len(matchedSingleManyArtists)))
    print("There are {0} unMatched many single Artists".format(len(unMatchedSingleManyArtists)))
    print("="*100)
    _, _ = clock("Last Run")
    
    return matchedChartResults, unMatchedSingleArtists, singleArtistAlbums, unMatchedSingleManyArtists, manySingleArtistAlbums

In [ ]:
known = {}
for name,results in bestRenames.items():
    print("del matchedChartResultsBillboard[\"{0}\"]".format(name))
    for v in results:
        if name == v:
            continue
        if known.get(v) is None:
            print("newRenames[\"{0}\"] = \"{1}\"".format(v,name))
            print("del matchedChartResultsBillboard[\"{0}\"]".format(v))
            if name.startswith("The "):
                print("newRenames[\"The {0}\"] = \"{0}\"".format(name))
            known[v] = True
    print("")

In [ ]:
def saveMatchedChartArtists(matchedChartResults, ctype=None):
    if ctype is None:
        ctype = chartType
    if ctype == "Billboard":
        print("Saving {0} artists to {1}".format(len(matchedChartResults), "matchedBillboardChartArtists.p"))
        saveFile(idata=matchedChartResults, ifile="matchedBillboardChartArtists.p", debug=True)
    elif ctype == "Top40":
        print("Saving {0} artists to {1}".format(len(matchedChartResults), "matchedTop40ChartArtists.p"))
        saveFile(idata=matchedChartResults, ifile="matchedTop40ChartArtists.p", debug=True)
    else:
        raise ValueError("Could not find data for chart type: {0}".format(ctype))


def getMatchedChartArtists(ctype=None, init=False):
    if ctype is None:
        ctype = chartType
    if ctype == "Billboard":
        matchedChartResults = getFile(ifile="matchedBillboardChartArtists.p", debug=True)
        print("There are {0} previously matched artists".format(len(matchedChartResults)))
    elif ctype == "Top40":
        matchedChartResults = getFile(ifile="matchedTop40ChartArtists.p", debug=True)
        #matchedChartResults = {}
        print("There are {0} previously matched artists".format(len(matchedChartResults)))
    else:
        raise ValueError("Could not find data for chart type: {0}".format(ctype))
        
    if init is True:
        print("You are initializing the matched chart artists!!!!")
        matchedChartResults = {}
    return matchedChartResults

In [ ]:
def getArtistAlbumData(ctype=None):
    if ctype is None:
        ctype = chartType
    if ctype == "Billboard":
        artistAlbumData = getFile(ifile="currentBillboardArtistAlbumData.p", debug=True)
    elif ctype == "Top40":
        artistAlbumData = getFile(ifile="currentTop40ArtistAlbumData.p", debug=True)
    else:
        raise ValueError("Could not find data for chart type: {0}".format(ctype))
    return artistAlbumData

def getFullChartArtistAlbumData(ctype=None):
    if ctype is None:
        ctype = chartType
    if ctype == "Billboard":
        fullChartData = getFile(ifile="currentBillboardFullChartArtistAlbumData.p", debug=True)
    elif ctype == "Top40":
        fullChartData = getFile(ifile="currentTop40FullChartArtistAlbumData.p", debug=True)
    else:
        raise ValueError("Could not find data for chart type: {0}".format(ctype))
    return fullChartData

In [ ]:
def updateMatchChartResults(latestResults, matchedChartResults):
    update=False
    nAdded=0
    for latestArtist,latestArtistResults in DataFrame(latestResults).to_dict().items():
        matches = sum([True for x in latestArtistResults.values() if x is not None])
        if matches == 0:
            continue
        if matchedChartResults.get(latestArtist) is None:
            pass
            update = True
            nAdded += 1
            #print("   Adding artist {0}".format(latestArtist))
            matchedChartResults[latestArtist] = latestArtistResults
        else:
            for db,dbID in latestArtistResults.items():
                if dbID is not None:
                    if matchedChartResults[latestArtist].get(db) is None:
                        pass
                        update = True
                        print(latestArtist)
                        print("      Setting {0} ID to {1}".format(db,dbID))
                        matchedChartResults[latestArtist][db] = str(dbID)
                    else:
                        if str(matchedChartResults[latestArtist][db]) != str(dbID):
                            raise ValueError("Error with db {0} for artist {1}, possibles [{2},{3}]".format(db, latestArtist, dbID, dbID, matchedChartResults[latestArtist][db]))
                            
    print("Added {0} new artists to the chart results".format(nAdded))
    return matchedChartResults, update

In [ ]:
for k,v in matchedRenames.items():
    print("{0}: {1}".format(k,v))

In [ ]:
del matchedChartResultsTop40["The Notorious B.I.G."]
del Ha Rule


In [ ]:
for i in range(100):
    print("renames[\"\"] = "")

In [ ]:

#    print(artist)
dbArtistData

In [ ]:
def updateDataFrame(dictVal):
    df = DataFrame(dictVal)
    dfT = df.T
    dfT["Matched"] = df.count(axis = 0)
    artistAlbums   = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in list(dfT.index)}
    artistAlbumsDF = DataFrame(artistAlbums).T
    dfT = dfT.join(artistAlbumsDF)
    
    print("Number of Entries in Stuff:         {0}".format((dfT.shape)))
    print("Number of Zero Matches in Stuff:    {0}".format((dfT[dfT["Matched"] == 0].shape)))
    print("Number of NonZero Matches in Stuff: {0}".format((dfT[dfT["Matched"] > 0].shape)))    
    print("")
    return dfT.copy(deep=True)

In [ ]:
def updateArtistAlumData():
    artistRenames = {}
    artistRenameData = getFile("renames2.yaml")
    print("There are {0} artist renames".format(len(artistRenameData)))
    
    artistAlbumData = getFile(ifile="currentArtistAlbumData.p", debug=True)
    print("There are {0} artist album entries".format(len(artistAlbumData)))
    
    for artist,rename in artistRenameData.items():
        if artistAlbumData.get(rename) is None and artistAlbumData.get(artist) is not None:
            artistAlbumData[rename] = artistAlbumData[artist]
            del artistAlbumData[artist]
            continue
            
        if artistAlbumData.get(rename) is not None and artistAlbumData.get(artist) is None:
            continue
            
        if artistAlbumData.get(rename) is None and artistAlbumData.get(artist) is None:
            #print("Neither {0} nor {1} exist".format(rename, artist))
            continue

        if artistAlbumData.get(rename) is not None and artistAlbumData.get(artist) is not None:
            tmp = artistAlbumData[artist] + artistAlbumData[rename]
            artistAlbumData[rename] = tmp
            print("Removing [{0}] and keeping [{1}]".format(artist, rename))
            del artistAlbumData[artist]
            
    print("There are {0} artist album entries".format(len(artistAlbumData)))    
    saveFile(idata=artistAlbumData, ifile="currentArtistAlbumData.p", debug=True)

In [ ]:
def removeUnmatchedEntries():
    dels = []
    for latestArtist in list(matchedChartResults.keys()):
        matches = sum([True for x in matchedChartResults[latestArtist].values() if x is not None])
        if matches == 0:
            dels.append(latestArtist)

    print(len(matchedChartResults))
    matchedChartResults = {k: v for k,v in matchedChartResults.items() if k not in dels}
    print(len(dels))
    print(len(matchedChartResults))
    saveFile(idata=matchedChartResults, ifile="matchedChartArtists.p", debug=True)

In [ ]:
for i,artist in enumerate(unMatchedSingleArtists + unMatchedSingleManyArtists):
    retvals = findNearest(artist, myArtists, num=2, cutoff=0.9)
    print(i,'\t',artist,'\t',retvals)

In [ ]:
mularts = getMA()
artist = "Drake And Future"
mularts.getArtistNames(artist)
mularts.delims

In [ ]:

print(fullChartData[artist])
print(artistAlbumData[artist])

In [ ]:
matchedChartResults[""]

# Rank Chart Data

In [ ]:
mmb = myMusicBase(debug=True)
mmb.findArtistAlbums(count=False)

In [ ]:
from listUtils import getFlatList
from searchUtils import findNearest
def getMatchedAlbums(artistName):
    artistAlbums = mmb.getArtistAlbumsByArtist(artistName)
    if artistAlbums.get("Match") is not None:
        matchedAlbums = getFlatList(artistAlbums["Match"].getAlbums().values())
        return matchedAlbums
    else:
        return None

def isKnownAlbum(artistName, albumName):
    albums  = getMatchedAlbums(artistName)
    if albums is None:
        return None
    matches = findNearest(albumName, albums, num=1, cutoff=0.8)
    if len(matches) == 1:
        return True
    return False

def isKnownArtist(artistName):
    allArtists = mmb.getArtists()
    matches    = findNearest(artistName, allArtists, num=1, cutoff=0.80)    
    if len(matches) == 1:
        return matches[0]
    return None

In [ ]:
from collections import Counter
albums = Counter()
songs  = Counter()

In [ ]:
def albumKey(artist, album):
    key = "   :::   ".join([artist, album])
    return key
    
def songKey(artist, song):
    key = "   :::   ".join([artist, song])
    return key

for artist, artistData in fullChartData.items():
    artistAlbums = artistData["Albums"]
    for album,albumData in artistAlbums.items():
        key = albumKey(artist, album)
        albums[key] = 0
        for chart,albumChartData in albumData.items():
            albums[key] += len(albumChartData.values())

In [ ]:
def findArtistRenames(albums):
    for item in albums.most_common():
        cnt = item[1]
        artist,album = item[0].split("   :::   ")
        knownArtist = isKnownArtist(artist)
        if knownArtist is None:
            continue
        if artist == knownArtist:
            continue
        print(": ".join([artist,knownArtist]))
findArtistRenames(albums)

In [ ]:
for item in albums.most_common():
    cnt = item[1]
    if cnt < 2:
        continue
    artist,album = item[0].split("   :::   ")
    knownArtist = isKnownArtist(artist)
    
    known = isKnownAlbum(artist,album)
    if known is None:
        known = "Get"
    else:
        if known is True:
            known = ""
        else:
            known = "--->"
    if len(known) > 0:
        print("{0: <5}{1: <8}{2: <35}{3: <50}".format(cnt,known,artist,album))

***
***

In [ ]:
for artist in matchedChartResults.keys():
    mb = matchedChartResults[artist]["AllMusic"]
    if mb is not None:
        try:
            int(mb)
        except:
            print(artist,mb)
            mb = str(mb[2:])
            print(artist,mb)
            matchedChartResults[artist]["AllMusic"] = mb

In [ ]:
print("Saving {0} artists to {1}".format(len(matchedChartResults), "matchedChartArtists.p"))
saveFile(idata=matchedChartResults, ifile="matchedChartArtists.p", debug=True)

In [ ]:
dfLatest = updateDataFrame(latestResults)
dfLatest
#dfTnew   = updateDataFrame(newSaveData)
#dfTnew

In [ ]:
latestResults

In [ ]:
dfT      = updateDataFrame(matchedChartResults)
dfLatest = updateDataFrame(latestResults)
dfTnew   = updateDataFrame(newSaveData)
_, _ = clock("Last Run")

In [ ]:
print("There are {0} updated matched artists".format(len(newSaveData)))
saveFile(idata=newSaveData, ifile="matchedChartArtists.p", debug=True)
_, _ = clock("Last Run")

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)
dfT = updateDataFrame(matchedChartResults)

results = dfT[dfT["Matched"] == 0]
print("There are {0} un matched entries.".format(results.shape[0]))
saveFile(idata=results, ifile="NoChartMatches.p", debug=True)
results = dfT[dfT["Matched"] > 0]
print("There are {0} matched entries.".format(results.shape[0]))
saveFile(idata=results, ifile="FewChartMatches.p", debug=True)
_, _ = clock("Last Run")

***
***
***

# Loop over unmatched artists

In [ ]:
unmatchedArtists = getFile(ifile="NoChartMatches.p", debug=True)
print("There are {0} previously unmatched artists".format(unmatchedArtists.shape))
unmatchedArtists

In [ ]:
for artist in unmatchedArtists.list:
    if artist.count("'") >= 2:
        print(artist)

In [ ]:
unmatchedArtistsToMatch = list(unmatchedArtists.index)
nProc = 8
tmp   = unmatchedArtistsToMatch
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
if len(retval) > 0:
    latestResults = dict(zip(tmp, retval))
else:
    latestResults = {}
newSaveData = {**matchedChartResults, **latestResults}
_, _ = clock("Last Run")

In [ ]:
dfT      = updateDataFrame(matchedChartResults)
dfLatest = updateDataFrame(latestResults)
dfTnew   = updateDataFrame(newSaveData)
_, _ = clock("Last Run")

In [ ]:
print("There are {0} updated matched artists".format(len(newSaveData)))
saveFile(idata=newSaveData, ifile="matchedChartArtists.p", debug=True)
_, _ = clock("Last Run")

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)
dfT      = updateDataFrame(matchedChartResults)
results = dfT[dfT["Matched"] == 0]
print("There are {0} un matched entries.".format(results.shape[0]))
saveFile(idata=results, ifile="NoChartMatches.p", debug=True)
results = dfT[dfT["Matched"] > 0]
print("There are {0} matched entries.".format(results.shape[0]))
saveFile(idata=results, ifile="FewChartMatches.p", debug=True)
_, _ = clock("Last Run")

***
***
***

# Test 'The'

In [ ]:
unmatchedArtists = getFile(ifile="NoChartMatches.p", debug=True)
print("There are {0} previously unmatched artists".format(unmatchedArtists.shape))
unmatchedArtists

In [ ]:
nProc = 8
tmp   = list(unmatchedArtists.index)
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProcThe(nProc, tmp)
elapsed(start, cmt)

In [ ]:
df = DataFrame(dict(zip(tmp, retval)))
dfT = df.T
dfT["Matched"] = df.count(axis = 0)
artistAlbums   = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in list(dfT.index)}
artistAlbumsDF = DataFrame(artistAlbums).T
dfT = dfT.join(artistAlbumsDF)

In [ ]:
theArtists = list(dfT[dfT["Matched"] > 0].index)
vals = {artist: "The {0}".format(artist) for artist in theArtists}
saveFile(idata=vals, ifile="tmp.yaml")

In [ ]:
# ELO

# Rename Code

In [ ]:
renames = {}

singleArtists = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) == 1]
result = {}
for artist in singleArtists:
    retvals = findNearest(artist, artistAlbumData.keys(), num=2, cutoff=0.95)
    if len(retvals) == 2:
        nretvals = [x for x in retvals if x != artist]
        num = len(artistAlbumData[artist])
        if num >= 2:
            print("{0: <40}{1: <10}{2}".format(artist,num,nretvals[0]))
            #print("renames[\"{0}\"] = \"\"".format(val))
            result[artist] = {"Num": num, "Possible": nretvals[0]}

In [ ]:
renames = {}
for artist in result.keys():
    possible = result[artist]["Possible"]
    if result.get(possible) is not None:
        if result[possible]["Num"] > result[artist]["Num"]:
            continue
    if renames.get(artist) is None:
        renames[artist] = []
    if possible not in renames[artist]:
        renames[artist].append(possible)
    #print("renames[\"{0}\"] = \"{1}\"".format(possible, artist))
saveFile(idata=renames, ifile="renames.yaml", debug=True)

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)

***
***
***

## Artist Renames

# Multi Artists Information

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)
from pandas import DataFrame
df = DataFrame(matchedChartResults)
dfT = df.T
print("Size of Artists:                        {0}".format(dfT.shape))
dfT["Matched"] = df.count(axis = 0)

results = dfT[dfT["Matched"] == 0]
saveFile(idata=results, ifile="NoChartMatches.p", debug=True)
results = dfT[dfT["Matched"] == 1]
saveFile(idata=results, ifile="FewChartMatches.p", debug=True)

print("Number of Zero Matches in New Stuff:    {0}".format((dfT[dfT["Matched"] == 0].shape)))
print("Number of NonZero Matches in New Stuff: {0}".format((dfT[dfT["Matched"] > 0].shape)))

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)
from pandas import DataFrame
df = DataFrame(matchedChartResults)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)

In [ ]:
renames = {}

singleArtists = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) == 1]
result = {}
for artist in singleArtists:
    retvals = findNearest(artist, artistAlbumData.keys(), num=2, cutoff=0.95)
    if len(retvals) == 2:
        nretvals = [x for x in retvals if x != artist]
        num = len(artistAlbumData[artist])
        if num >= 2:
            print("{0: <40}{1: <10}{2}".format(artist,num,nretvals[0]))
            #print("renames[\"{0}\"] = \"\"".format(val))
            result[artist] = {"Num": num, "Possible": nretvals[0]}

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)
from pandas import DataFrame
df = DataFrame(matchedChartResults)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)


artistAlbums   = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in list(dfT.index)}
artistAlbumsDF = DataFrame(artistAlbums).T
dfT = dfT.join(artistAlbumsDF)
print("Size of Chart Aritsts:                  {0}".format(dfT.shape))


results = dfT[dfT["Matched"] == 0]
saveFile(idata=results, ifile="NoChartMatches.p", debug=True)
results = dfT[dfT["Matched"] == 1]
saveFile(idata=results, ifile="FewChartMatches.p", debug=True)

print("Number of Zero Matches in New Stuff:    {0}".format((dfT[dfT["Matched"] == 0].shape)))
print("Number of NonZero Matches in New Stuff: {0}".format((dfT[dfT["Matched"] > 0].shape)))

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)
from pandas import DataFrame
df = DataFrame(matchedChartResults)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)


artistAlbums   = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in list(dfT.index)}
artistAlbumsDF = DataFrame(artistAlbums).T
dfT = dfT.join(artistAlbumsDF)
print("Size of Chart Aritsts:                  {0}".format(dfT.shape))


results = dfT[dfT["Matched"] == 0]
saveFile(idata=results, ifile="NoChartMatches.p", debug=True)
results = dfT[dfT["Matched"] == 1]
saveFile(idata=results, ifile="FewChartMatches.p", debug=True)

print("Number of Zero Matches in New Stuff:    {0}".format((dfT[dfT["Matched"] == 0].shape)))
print("Number of NonZero Matches in New Stuff: {0}".format((dfT[dfT["Matched"] > 0].shape)))

In [ ]:
notAnArtist = ["Soundtrack", 'Various', 'Varios']
noMatches = getFile("NoChartMatches.p")
noMatches.sort_values("Num", ascending=False, inplace=True)
ranking = noMatches[["Num", "Albums"]]
ranking = ranking[~ranking.index.isin(notAnArtist)]
toget = list(ranking[ranking["Num"] > 1].index)
print("Here are {0} artists to get".format(len(toget)))
saveFile(idata=toget, ifile="../discogs/unmatchedArtists4.p")

# Merge No Matches

In [ ]:
noChartMatches = getFile("NoChartMatches.p")
len(noChartMatches)

In [ ]:
x = "The Hello"
x[4:]

# Pool/Process

In [ ]:
latestResults = dict(zip(tmp, retval))

In [ ]:
from pandas import DataFrame
df = DataFrame(matchedChartResults)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)
print("Number of Zero Matches: {0}".format((dfT[dfT["Matched"] == 0].shape)))

dfLatest = DataFrame(latestResults)
dfLatestT = dfLatest.T
dfLatestT["Matched"] = dfLatest.count(axis = 0)
print("Number of Zero Matches in New Stuff: {0}".format((dfLatestT[dfLatestT["Matched"] == 0].shape)))
print("Number of NonZero Matches in New Stuff: {0}".format((dfLatestT[dfLatestT["Matched"] > 0].shape)))

In [ ]:
newSaveData = {**matchedChartResults, **latestResults}

df = DataFrame(newSaveData)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)
print("Number of Zero Matches in Updated Stuff: {0}".format((dfT[dfT["Matched"] == 0].shape)))
print("Number of NonZero Matches in Updated Stuff: {0}".format((dfT[dfT["Matched"] > 0].shape)))

results = dfT[dfT["Matched"] == 0]
saveFile(idata=results, ifile="NoChartMatches.p", debug=True)
results = dfT[dfT["Matched"] == 1]
saveFile(idata=results, ifile="FewChartMatches.p", debug=True)

print("Number of Zero Matches in New Stuff:    {0}".format((dfT[dfT["Matched"] == 0].shape)))
print("Number of NonZero Matches in New Stuff: {0}".format((dfT[dfT["Matched"] > 0].shape)))

_, _ = clock("Last Run")

In [ ]:
saveFile(idata=newSaveData, ifile="matchedChartArtists.p", debug=True)
_, _ = clock("Last Run")

# Multi Artists

In [ ]:
manyArtists   = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) > 1]
manyArtistAlbums = {artist: {"Artists": mularts.getArtistNames(artist)} for artist in manyArtists}

In [ ]:
DataFrame(manyArtistAlbums).T.head()

In [ ]:
from collections import Counter
cntr = Counter()
for k,artists in manyArtistAlbums.items():
    for artist in artists["Artists"]:
        cntr[artist] += 1

In [ ]:

artistAlbums   = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in list(dfT.index)}

In [ ]:
from pandas import Series
multis = Series(cntr).sort_values(ascending=False).to_dict()
multis = {k: {"Chart Counts": v, "Num": artistAlbumData.get(k)} for k,v in multis.items()}
for k in multis.keys():
    if multis[k]["Num"] is not None:
        multis[k]["Num"] = len(multis[k]["Num"])
    else:
        multis[k]["Num"] = 0

multisDF = DataFrame(multis).T
noMultis = multisDF[multisDF["Num"] == 0]

print("Here are {0} artists to get".format(len(noMultis.index)))
saveFile(idata=list(noMultis.index), ifile="../discogs/unmatchedArtists_Multi.p")


In [ ]:
noMultis

In [ ]:
singleArtists = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) == 1]
artistAlbums  = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in singleArtists}
len(artistAlbums)

In [ ]:
manyArtists   = [artist for artist in list(artistAlbumData.keys()) if len(mularts.getArtistNames(artist)) > 1]
manyArtistAlbums = {artist: {"Num": len(artistAlbumData[artist]), "Albums": artistAlbumData[artist]} for artist in manyArtists}
len(manyArtistAlbums)

In [ ]:
from pandas import DataFrame, Series
nAlbumsDF = DataFrame(artistAlbums).T
#nAlbumsDF.columns = ["Albums"]
nAlbumsDF.head()

***
***

# Analyze ManyArtists

In [ ]:
from pandas import DataFrame, Series
nManyAlbumsDF = DataFrame(manyArtistAlbums).T
nManyAlbumsDF = nManyAlbumsDF.sort_values("Num", ascending=False)
#nAlbumsDF.columns = ["Albums"]
print("\n".join(list(nManyAlbumsDF[nManyAlbumsDF["Num"] >= 4].index)))

# Match The Single Artists

In [ ]:
prevResults = {}
prevResults = getFile(ifile="matchedChartArtists.p", debug=True)

maxI = 4500
if maxI == 0:
    prevResults = {}
print("Found {0} previously matched artists".format(len(prevResults)))

In [ ]:
nProc = 8
tmp   = singleArtists[8000:14000]
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
savedata2 = dict(zip(tmp, retval))
len(savedata2)

In [ ]:
oldSaveData = getFile(ifile="matchedChartArtists.p", debug=True)
saveFile(idata=oldSaveData, ifile="matchedChartArtists_tmp.p", debug=True)

print("Num Old Data:  {0}".format(len(oldSaveData)))
print("Num Save Data: {0}".format(len(savedata2)))
newSaveData = {**oldSaveData, **savedata2}
print("Num New Data:  {0}".format(len(newSaveData)))

In [ ]:
#saveFile(idata=newSaveData, ifile="matchedChartArtists.p", debug=True)
oldSaveData = getFile(ifile="matchedChartArtists.p", debug=True)
saveFile(idata=oldSaveData, ifile="matchedChartArtists_tmp.p", debug=True)

# Load All Matched Single Chart Artists

In [ ]:
maindb = mainDB(create=False)

In [ ]:

dfT = dfT.join(nAlbumsDF)
dfT.head()

In [ ]:
noMatches = getFile("NoChartMatches.p")
noMatches.sort_values("Num", ascending=False, inplace=True)
ranking = noMatches[["Num", "Albums"]]
ranking = ranking[~ranking.index.isin(notAnArtist)]mcm
toget = list(ranking[ranking["Num"] > 2].index)
saveFile(idata=toget, ifile="../discogs/unmatchedArtists2.p")

In [ ]:
ranking[ranking.index.str.contains("feat")]

In [ ]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()

In [ ]:
from pandas import DataFrame
df = DataFrame(matchedChartResults)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)
dfT.head()

# Loop Over The No Matches

In [ ]:
unMatchedArtists = getFile(ifile="NoChartMatches.p", debug=True)

from time import sleep
from copy import deepcopy

mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()

mcm = matchChartMusic(mdb)
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
multiDelimArtists=open("../multiartist/multiDelimArtists.dat").readlines()
multiDelimArtists = [x.replace("\n", "") for x in multiDelimArtists]
multiDelimArtists[:10]
mularts.setKnownMultiDelimArtists(multiDelimArtists)
results = {}

In [ ]:
def process(artist):
    albums = artistAlbumData[artist]
    mcm.setChartArtistData(artist, albums)
    retval = mcm.matchChartArtist(ratioCut=0.8, returnData=True)
    return retval

def poolProc(nProc, tmp):
    pool = Pool(processes=nProc)
    result = pool.map_async(process, tmp)

    while not result.ready():
        sleep(10)
        print("")
        return result.get()

nProc = 8
tmp   = unMatchedArtists[:4]
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
matches = dict(zip(tmp, retval))

In [ ]:
matchedChartResults = getFile(ifile="matchedChartArtists.p", debug=True)

In [ ]:
for artist in matches.keys():
    matchedChartResults[artist] = matches[artist]
saveFile(idata=matchedChartResults, ifile="matchedChartArtists.p", debug=True)

In [ ]:


from pandas import DataFrame
df = DataFrame(matches)
df
dfT = df.T
dfT["Matched"] = df.count(axis = 0)
dfT.head()
dfT[dfT["Matched"] > 0]

# Find Errors and Merge DBs

In [ ]:
errors = {}
print("{0: <20}{1: <15}{2: <40}{3: <40}".format("Artist", "DB", "My DB ID", "Chart DB ID", "ERROR"))
for i,artist in enumerate(list(df.columns)):
    print("{0: <20}".format(artist), end="")
    retval = mdb.getArtistDataIDs(artist)
    firstDB = True
    for db,dbID in retval.items():
        if firstDB is True:
            firstDB = False
        else:
            print("{0: <20}".format(''), end="")
        print("{0: <15}".format(db), end="")
            
        if dbID is None:
            print("{0: <40}".format(''), end="")
        else:
            print("{0: <40}".format(dbID), end="")

        chartDBID = df[artist][db]
        if chartDBID is None:
            print("{0: <40}".format(''), end="")
        else:
            print("{0: <40}".format(chartDBID), end="")
            
        if chartDBID is not None and dbID is None:
            mdb.add(artist, db, chartDBID)
            
        if all([dbID,chartDBID]):
            if dbID == chartDBID:
                pass
            else:
                print(dbID==chartDBID, end="")
                #mdb.add(artist, db, chartDBID)
                if errors.get(artist) is None:
                    errors[artist] = {}
                errors[artist][db] = {"Mine": dbID, "Chart": chartDBID}
        print("")
    print("")
    if len(errors) > 50:
        break

In [ ]:
#maindb.dbdata["LastFM"]["Utils"].getArtistID('Morten Harket')
#mdb.add("Blackstreet", "MusicBrainz", "78850123839630113234659525690757885261")
#mdb.add("214847928836161113317702378728828431474
#mdb.save()

In [ ]:

nProc = 4
tmp   = deepcopy(singleArtists[:80])
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
nProc = 4
tmp   = deepcopy(singleArtists[:160])
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
nProc = 4
tmp   = deepcopy(singleArtists[:640])
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
nProc = 8
tmp   = deepcopy(singleArtists[:640])
start, cmt = clock("Trying N={0} and L={1}".format(nProc, len(tmp)))
retval = poolProc(nProc, tmp)
elapsed(start, cmt)

In [ ]:
mcm = matchChartMusic(mdb)
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
results = {}


n = len(artistAlbumData)
from collections import Counter
multiCntr = Counter()
multis = {}

for i,(artist,albums) in enumerate(artistAlbumData.items()):
    if i <= maxI:
        continue
    matches = mularts.getArtistNames(artist)
    if len(matches) > 1:
        print("{0} -> {1}".format(artist,matches))
        for match in matches:
            multiCntr[match] += 1
        multiCntr[artist] += 1
        continue
    #print("\n")
    #print("="*50)
    print(i,'/',n,"\t{0: <40}".format(artist),'\t',len(albums))
    mcm.setChartArtistData(artist, albums)
    mcm.matchChartArtist(ratioCut=0.85)
    
    
    if i % 250 == 0:
        if len(prevResults) > 0:
            results = {**prevResults, **mcm.getResults()}
            prevResults = {}
        else:
            results = mcm.getResults()
        print("Saving {0} artists".format(len(results)))
        saveFile(idata=results, ifile="matchedChartArtists.p", debug=True)

In [ ]:
list(artistAlbumData.keys())[maxI:]

In [ ]:

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        dbdata["RateYourMusic"]["Artists"].parseDownloadedFiles()
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

In [ ]:
multiCntr.most_common()

In [ ]:
from pandas import DataFrame
df = DataFrame(results)
dfT = df.T
dfT["Matched"] = df.count(axis = 0)
dfT[dfT["Matched"] == 0]

In [ ]:
for artist,artistData in fullChartData.items(): #[artist][key][album][chartName][date] = i
    print("\n")
    print("="*100)
    print(artist)
    # Songs
    print("\tSongs:")
    for songName,songData in artistData["Songs"].items():
        print("\t",songName,'\t\t',end="")
        chartWeeks = 0
        for chartName,songChartData in songData.items():
            weeks = len(songChartData)
            chartWeeks += weeks
        print(chartWeeks)
        
    # Albums
    print("\tAlbums:")
    for albumName,albumData in artistData["Albums"].items():
        print("\t",albumName,'\t\t',end="")
        chartWeeks = 0
        for chartName,albumChartData in albumData.items():
            weeks = len(albumChartData)
            chartWeeks += weeks
        print(chartWeeks)

In [ ]:
knownArtists

In [ ]:
from fileUtils import getBaseFilename
results = {}
for ifile in files:
    chartData = getFile(ifile)
    cname     = getBaseFilename(ifile)
    print("{0: <40}".format(cname),end="")
    for date, values in chartData.items():
        for i,item in enumerate(values):
            artist = item["Artist"]
            prevArtist = None
            if multiRenames.get(artist) is not None:
                prevArtist = artist
                artist = multiRenames[artist]
            if singleRenames.get(artist) is not None:
                idx    = str(singleRenames[artist])
                artist = artistIDToName[idx]
            if prevArtist is None:
                prevArtist = artist
            album  = item["Album"]
            if results.get(artist) is None:
                results[artist] = {"Songs": {}, "Albums": {}}
            if cname.endswith("Albums"):
                key = "Albums"
            else:
                key = "Songs"
            if results[artist][key].get(album) is None:
                results[artist][key][album] = {}
            if results[artist][key][album].get(cname) is None:
                results[artist][key][album][cname] = {}
            results[artist][key][album][cname][date] = i
    print(len(results))

In [ ]:
for chartName, chartData in newCharts.items():
    

In [ ]:
def getStarterFiles(charts):
    for cid, cname in charts.items():
        print("{0} --> {1}".format(cid,cname))
        url      = "https://top40-charts.com/chart.php?cid={0}".format(cid)
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,}

        savedir  = join(basedir, "data", "top40")
        mkDir(savedir)

        savename = setFile(savedir, "{0}.p".format(cname.replace("/", " ")))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        chartsToGet = {}
        bsdata = getHTML(data)
        select = bsdata.find("select", {"name": "date"})
        if select is None:
            raise ValueError("No dates!")
        for option in select.findAll("option"):
            attr  = option.attrs
            value = attr['value']
            url   = "https://top40-charts.com/chart.php?cid={0}&date={1}".format(cid, value)
            chartsToGet[value] = url

        savename = setFile(savedir, "{0}-Dates.p".format(cname.replace("/", " ")))
        print("Saving {0} dates to {1}".format(len(chartsToGet), savename))
        saveFile(idata=chartsToGet, ifile=savename)
        sleep(2)

In [ ]:
from datetime import date, timedelta

def all_sundays(year):
    # January 1st of the given year
       dt = date(year, 1, 1)
        # First Sunday of the given year       
       dt += timedelta(days = 6 - dt.weekday())  
       while dt.year == year:
          yield dt
          dt += timedelta(days = 7)
            
from pandas import DataFrame
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
from difflib import SequenceMatcher

def getRowByIndex(pdf, idx):
    return pdf.loc[idx]

In [ ]:
def getBestArtistIDMatch(artistName, artistResults, possibleMatches, N=3, cutoff=0.7, debug=False):    
    myAlbumNames = list(artistResults)
    idxResults = {}
    matchValue = {}
    for idx in possibleMatches:
        idxResults[idx] = 0
        matchValue[idx] = artistName

        artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
        artistAlbums     = artistAlbumsData["Albums"]

        for myAlbumName in myAlbumNames:
            maxRatio = 0
            for albumType, albumTypeData in artistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                    ratio = s.ratio()
                    if ratio > maxRatio:
                        maxRatio = ratio

            if maxRatio > cutoff:
                idxResults[idx] += maxRatio

        if debug:
            print("\t{0: <5}{1: <15}{2}".format(idxResults[idx], idx, matchValue[idx]))
            
    if len(idxResults) > 0:
        mc     = Counter(idxResults)
        best   = mc.most_common(1)[0]
        idx    = best[0]
        ratio  = round(best[1],2)
        artist = matchValue[idx]
        return (idx, artist, ratio)
    else:
        return (None, None, None)
    


def getBestArtistMatch(artistName, artistResults, N=3, cutoff=0.7, debug=False):
    myAlbumNames = list(artistResults)
    if debug:
        print(artistName)

    idxResults = {}
    matchValue = {}

    
    for artist in findNearest(artistName, artistNameToID.keys(), N, cutoff):
        for idx in artistNameToID[artist]:
            idxResults[idx] = 0
            matchValue[idx] = artist
            
            artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
            artistAlbums     = artistAlbumsData["Albums"]
            
            for myAlbumName in myAlbumNames:
                maxRatio = 0
                for albumType, albumTypeData in artistAlbums.items():
                    for albumID, dbAlbumName in albumTypeData.items():
                        s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                        ratio = s.ratio()
                        if ratio > maxRatio:
                            maxRatio = ratio
                            
                if maxRatio > cutoff:
                    idxResults[idx] += maxRatio
            
            if debug:
                print("\t{0: <5}{1: <15}{2}".format(idxResults[idx], idx, matchValue[idx]))

    if len(idxResults) > 0:
        mc     = Counter(idxResults)
        best   = mc.most_common(1)[0]
        idx    = best[0]
        ratio  = round(best[1],2)
        artist = matchValue[idx]
        return (idx, artist, ratio)
    else:
        return (None, None, None)

# Discogs Downloads

In [ ]:
disc = discogs()
discdf = disc.getMasterSlimArtistDiscogsDB()
artistIDToName = discdf["DiscArtist"].to_dict()
from masterdb import getArtistAlbumsDB, discConv
artistAlbumsDB = getArtistAlbumsDB(disc)

artistNameToID = {}
print("Found {0} ID -> Name entries".format(len(artistIDToName)))
for artistID,artistName in artistIDToName.items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToID)))
mulArts  = multiArtist(cutoff=0.9, discdata=artistNameToID, exact=False)

    matches = mulArts.getArtistNames(artist)
    if len(matches) > 1:
        continue

# Rename Files

In [ ]:
singleRenames = getFile(ifile="singleRenames.p")
multiRenames  = getFile(ifile="multiRenames.p")
knownArtists  = getFile(ifile="artistMap.p")

for k in singleRenames.keys():
    v = singleRenames[k]
    if isinstance(v, list):
        if len(v) == 2:
            singleRenames[k] = v[0]
saveFile(idata=singleRenames, ifile="singleRenames.p")

# Get Starter File

In [ ]:
savename=join(basedir, "data", "top40", "starter.p")

def getStarter(basedir):
    url="https://top40-charts.com"
    getWebData(base=url, savename=savename, useSafari=False)
    
if not isFile(savename):
    getStarter(basedir)
    fdata = getHTML(savename)
else:
    fdata = getHTML(savename)

In [ ]:
fdata

In [ ]:
charts = {}
for iform,formdata in enumerate(fdata.findAll("form")):
    print(iform)
    for isel,seldata in enumerate(formdata.findAll("select", {"name": "cid"})):
        print(isel)
        for iop,opdata in enumerate(seldata.findAll("option")):
            attrs  = opdata.attrs
            value  = attrs['value']
            charts[value] = opdata.text
#return charts
charts

In [ ]:
chartData  = {}
dirname = None
baseURL = "http://top40-charts.com"

## Get Charts
def getCharts(fdata):
    charts = {}
    for iform,formdata in enumerate(fdata.findAll("form")):
        for isel,seldata in enumerate(formdata.findAll("select", {"name": "cid"})):
            for iop,opdata in enumerate(seldata.findAll("option")):
                attrs  = opdata.attrs
                value  = attrs['value']
                charts[value] = opdata.text
    return charts


## Get Dates
def getDates(fdata):
    dates = []
    for iform,formdata in enumerate(fdata.findAll("form")):
        for isel,seldata in enumerate(formdata.findAll("select", {"name": "date"})):
            continue
            for iop,opdata in enumerate(seldata.findAll("option")):
                attrs  = opdata.attrs
                value  = attrs['value']
                dates.append(value)
    dates = sorted(list(set(dates)))
    return dates

charts = getCharts(fdata)

In [ ]:
charts

### Last Downloaded on 5/21/20

In [ ]:
def getStarterFiles(charts):
    for cid, cname in charts.items():
        print("{0} --> {1}".format(cid,cname))
        url      = "https://top40-charts.com/chart.php?cid={0}".format(cid)
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,}

        savedir  = join(basedir, "data", "top40")
        mkDir(savedir)

        savename = setFile(savedir, "{0}.p".format(cname.replace("/", " ")))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        chartsToGet = {}
        bsdata = getHTML(data)
        select = bsdata.find("select", {"name": "date"})
        if select is None:
            raise ValueError("No dates!")
        for option in select.findAll("option"):
            attr  = option.attrs
            value = attr['value']
            url   = "https://top40-charts.com/chart.php?cid={0}&date={1}".format(cid, value)
            chartsToGet[value] = url

        savename = setFile(savedir, "{0}-Dates.p".format(cname.replace("/", " ")))
        print("Saving {0} dates to {1}".format(len(chartsToGet), savename))
        saveFile(idata=chartsToGet, ifile=savename)
        sleep(2)

In [ ]:
getStarterFiles(charts)

# Download Chart Data

In [ ]:
for cid, cname in charts.items():
    print("{0} --> {1}".format(cid,cname))
    
    
    cidsavedir = join(basedir, "data", "top40")
    savename   = setFile(cidsavedir, "{0}-Dates.p".format(cname.replace("/", " ")))
    chartData  = getFile(savename)
    
    values = sorted(chartData.keys(), reverse=True)
    for idts,value in enumerate(values):
        if value is None:
            continue
            
        url = chartData[value]
        try:
            if getDateTime(value).year < 2000:
                continue
        except:
            continue

        try:
            if getDateTime(value).year < 201:
                continue        
        except:
            continue
            
        if value is None:
            continue
            
        savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
        savename = setFile(savedir, "{0}.p".format(value))
        if savename is None:
            continue
            
        if isFile(savename):
            continue
            print("Touching {0}".format(savename))
            Path(savename).touch()
        else:
            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers={'User-Agent':user_agent,}
    
            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need

            print("URL ---> {0}".format(url))
            print(idts,'/',len(values),"\tSaving {0}".format(savename))
            saveJoblib(data=data, filename=savename, compress=True)
            sleep(2)

# Parse Charts

In [ ]:
def getChartData(chartData, debug=False):
    results = []
    pos = 1
    
    debVars = None
    
    for it,table in enumerate(chartData.findAll("table")):
        ths = table.findAll("th")
        trs = table.findAll("tr")
        attrs = table.attrs
        if debug:
            print(it,len(ths),len(trs),attrs)

        if attrs == {'cellpadding': '0', 'cellspacing': '0', 'borer': '0'}:
            if len(trs) == 1:
                tds = trs[0].findAll("td")
                if len(tds) == 3:
                    refs = tds[2].findAll("a")
                    if len(refs) == 2:
                        album  = refs[0].text
                        artist = refs[1].text
                        results.append({"Artist": artist, "Album": album})
                        if debug:
                            print(pos,'\t',artist,'\t',album)
                        pos += 1

    return results

In [ ]:
for cid, cname in charts.items():
    print("{0} --> {1}".format(cid,cname))
    
    
    cidsavedir = join(basedir, "data", "top40")
    savename   = setFile(cidsavedir, "{0}-Dates.p".format(cname.replace("/", " ")))
    chartData  = getFile(savename)
    
    values = sorted(chartData.keys(), reverse=True)
    chartResults = {}
    for idts,value in enumerate(values):
        if value is None:
            continue
            
        savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
        savename = setFile(savedir, "{0}.p".format(value))            
        data     = getHTML(savename)
        results  = getChartData(data)
        chartResults[value] = results
        if len(results) == 0:
            print('\t',cname,'\t\t',value,'\t',len(results)," <<-------")
        else:
            print('\t',cname,'\t\t',value,'\t',len(results))
            
    savedir = join(basedir, "results", "top40")
    savename = setFile(savedir, "{0}.p".format(cname.replace("/", " ")))
    saveJoblib(data=chartResults, filename=savename, compress=True)
    #savename = setFile("~/Dropbox/charts", "{0}.p".format(cname.replace("/", " ")))
    #saveJoblib(data=chartResults[cid], filename=savename, compress=True)

# Aggregate Charts

In [ ]:
from searchUtils import findExt
savedir = join(basedir, "results", "top40")
files = findExt(savedir, ".p")
print("Found {0} files".format(len(files)))

In [ ]:
from fileUtils import getBaseFilename
results = {}
for ifile in files:
    chartData = getFile(ifile)
    cname     = getBaseFilename(ifile)
    print("{0: <40}".format(cname),end="")
    for date, values in chartData.items():
        for i,item in enumerate(values):
            artist = item["Artist"]
            prevArtist = None
            if multiRenames.get(artist) is not None:
                prevArtist = artist
                artist = multiRenames[artist]
            if singleRenames.get(artist) is not None:
                idx    = str(singleRenames[artist])
                artist = artistIDToName[idx]
            if prevArtist is None:
                prevArtist = artist
            album  = item["Album"]
            if results.get(artist) is None:
                results[artist] = {"Songs": {}, "Albums": {}}
            if cname.endswith("Albums"):
                key = "Albums"
            else:
                key = "Songs"
            if results[artist][key].get(album) is None:
                results[artist][key][album] = {}
            if results[artist][key][album].get(cname) is None:
                results[artist][key][album][cname] = {}
            results[artist][key][album][cname][date] = i
    print(len(results))

In [ ]:
saveFile(ifile="chartResults.p", idata=results, debug=True)

In [ ]:
from collections import Counter
slimResults = Counter()
for artist, artistData in results.items():
    for key, keyData in artistData.items():
        for album, albumData in keyData.items():
            slimResults[artist] += sum({k: len(v) for k,v in albumData.items()}.values())

In [ ]:
saveFile(ifile="chartCounter.p", idata=slimResults, debug=True)

In [ ]:
for artist, artistData in results.items():
    print(artist)
    print("    Songs:")
    for album, albumData in artistData["Songs"].items():
        print('\t',album,'\t',len(albumData))
        for chart, chartData in albumData.items():
            pass
            #print('\t\t',chart,chartData)
    print("    Albums:")
    for album, albumData in artistData["Albums"].items():
        print('\t',album,'\t',len(albumData))
        for chart, chartData in albumData.items():
            pass
            #print('\t\t',chart,chartData)
    break

# Find Missing Artists

In [ ]:
slimResults = getFile(ifile="chartCounter.p", debug=True)
fullResults = getFile(ifile="chartResults.p", debug=True)

singleRenames = getFile(ifile="singleRenames.p", debug=True)
multiRenames  = getFile(ifile="multiRenames.p", debug=True)
knownArtists  = getFile(ifile="artistMap.p", debug=True)

In [ ]:
skips = {}

# Unknown Artists

In [ ]:
## 5500
artistsToGet = {}
#for i, (artist, cnt) in enumerate(slimResults.most_common()):
for i, (artist, cnt) in enumerate(sorted(slimResults.items(), key=lambda pair: pair[1], reverse=True)):
    if i <= 14961:
        continue
    if i % 250 == 0:
        print("==>",i,len(slimResults))
    if multiRenames.get(artist) is not None:
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = singleRenames[artist]
        artist = artistIDToName[idx]
    if knownArtists.get(artist) is not None:
        continue
    if skips.get(artist) is not None:
        continue

    matches = mulArts.getArtistNames(artist)
    if len(matches) > 1:
        continue

    mdata = getMusicData("DiscArtist", artist)
    if mdata is None:
        artistsToGet[artist] = cnt
        print("{0: <6}{1: <40}{2: <6}{3}".format(i,artist,cnt,len(artistsToGet)))
        if len(artistsToGet) > 200:
            break
    elif isinstance(mdata, DataFrame):
        if mdata.shape[0] >= 1:
            continue
        artistsToGet[artist] = cnt
        print("{0: <6}{1: <40}{2: <6}{3}".format(i,artist,cnt,len(artistsToGet)))
        if len(artistsToGet) > 200:
            break

In [ ]:
from collections import Counter
cutoff = 0.7
skips = {}
possibleResults = {}
for i,(artist, cnt) in enumerate(artistsToGet.items()):
    print(i,"/",len(artistsToGet),'  \t',artist)
    artistAlbums = set(list(fullResults[artist]["Songs"].keys()) + list(fullResults[artist]["Albums"].keys()))
    results      = getBestArtistMatch(artist, artistAlbums, N=3, cutoff=cutoff)
    if results[2] is not None:
        if results[2] >= cutoff:
            print(i,"/",len(artistsToGet),'  \t',artist,results)
            possibleResults[artist] = list(results[:-1])
        else:
            skips[artist] = True
    else:
        skips[artist] = True

In [ ]:
keep = {}

#mdata = getMusicData("DiscArtist", "Tyrese")
#keep["Tyrese"] = [mdata.index[0], mdata["Name"].values[0]]
#keep["PPK \r"] = [mdata.index[0], mdata["Name"].values[0]]

#keep['Sarah Brightman']=['59756', 'Sarah Brightman']

for artist,results in possibleResults.items():
    keep[artist] = results
#keep["Lil' Romeo"] = ['134987', "Lil' Romeo"]
keep

In [ ]:
singleRenames = getFile(ifile="singleRenames.p", debug=True)
for k in keep.keys():
    v = keep[k]
    if isinstance(v, list):
        if len(v) == 2:
            singleRenames[k] = v[0]
            print(k,v[0])
saveFile(idata=singleRenames, ifile="singleRenames.p")

In [ ]:
saveFile(idata=skips, ifile="toget11.p")

# Get Multi Results

In [ ]:
from collections import Counter
cutoff = 0.8
multiMatchResult = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
    if i > 0 and i % 1000 == 0:
        print("Passed",i,'/',len(slimResults))
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    if knownArtists.get(prevArtist) is not None:
        continue
    mdata = getMusicData("DiscArtist", prevArtist)
    if isinstance(mdata, DataFrame):
        if mdata.shape[0] <= 1:
            continue
        matches = mdata["Name"].index
        artistAlbums = set(list(fullResults[prevArtist]["Songs"].keys()) + list(fullResults[prevArtist]["Albums"].keys()))
        results = getBestArtistIDMatch(artist, artistAlbums, matches, N=3, cutoff=cutoff)
        if results[2] is not None:
            if results[2] >= cutoff:
                print(i,"/",len(slimResults),'  \t',artist,results)
                multiMatchResult[artist] = list(results[:-1])

In [ ]:
print(len(knownArtists))
knownArtists.update(multiMatchResult)
print(len(knownArtists))
saveFile(idata=knownArtists, ifile="artistMap.p", debug=True)

In [ ]:
multiMatchResult[artist] = results

# Get Multi Artist Results

In [ ]:
cutoff = 0.8
multiResult = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
    if i > 0 and i % 1000 == 0 or i == 100:
        print("Passed",i,'/',len(slimResults))
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    artistAlbums = set(list(fullResults[prevArtist]["Songs"].keys()) + list(fullResults[prevArtist]["Albums"].keys()))
    matches = mulArts.getArtistNames(artist)
    if len(matches) == 1:
        continue
    for match in matches:
        if singleRenames.get(match) is not None:
            idx    = str(singleRenames[match])
            #print('\t',match,' --> ',end="")
            match  = artistIDToName[idx]  
            #print(match)
        mdata = getMusicData("DiscArtist", match)
        if not isinstance(mdata, DataFrame):
            results = getBestArtistMatch(match, artistAlbums, N=3, cutoff=cutoff)
            if results[2] is not None:
                if results[2] >= cutoff:
                    print(i,"/",len(slimResults),'  \t',artist,results)
                    if multiResult.get(artist) is None:
                        multiResult[artist] = {"Results": results, "Match": []}
                    multiResult[artist]["Match"].append(match)

    if i > 5000:
        break

In [ ]:
for k,v in multiResult.items():
    x = k.replace(v["Match"][0], v["Results"][1])
    print("multiRenames[\"{0}\"] = \"{1}\"".format(k, x))

In [ ]:
multiRenames = getFile("multiRenames.p")
print(len(multiRenames))
multiRenames["Drake, Wizkid & Kyla"] = "Drake, WizKid & Kyla"
multiRenames["A. R. Rahman & Pussycat Dolls"] = "A.R. Rahman & Pussycat Dolls"
multiRenames["Calvin Harris, Pharell Williams, Katy Perry & Big Sean"] = "Calvin Harris, Pharrell Williams, Katy Perry & Big Sean"
multiRenames["Flo Rida, Sage The Gemini & Lookas"] = "Flo Rida, Sage The Gemini & LooKas"
multiRenames["DJ Snake & Lil Jon"] = "DJ Snake & Lil' Jon"
multiRenames["Rihanna & Jay-z"] = "Rihanna & Jay-Z"
multiRenames["Lil Jon & East Side Boyz"] = "Lil' Jon & East Side Boyz"
multiRenames["Remady & Manu-L"] = "DJ Remady & Manu-L"
multiRenames["Lil Jon & The East Side Boyz, Usher & Ludacris"] = "Lil' Jon & The East Side Boyz, Usher & Ludacris"
multiRenames["Jay Sean, Sean Paul & Lil Jon"] = "Jay Sean, Sean Paul & Lil' Jon"
multiRenames["Brandy & Ray Jay"] = "Brandy & Ray J"
multiRenames["KitschKrieg, Trettmann, Gringo, Ufo361 & Gzuz"] = "Kitschkrieg, Trettmann, Gringo, Ufo361 & Gzuz"
multiRenames["Mike WiLL Made-It, Miley Cyrus, Wiz Khalifa & Juicy J"] = "Mike WiLL Made It, Miley Cyrus, Wiz Khalifa & Juicy J"
multiRenames["DJ Broiler & Ravvel"] = "Broiler & Ravvel"
multiRenames["Bob Sinclar & Raffaella Carra"] = "Bob Sinclar & Raffaella Carrà"
multiRenames["Revolverheld & Marta Jandova"] = "Revolverheld & Marta Jandová"
multiRenames["Parra For Cuva & Anna Naklab"] = "Parra for Cuva & Anna Naklab"
multiRenames["Patti Labelle & Ron Isley"] = "Patti LaBelle & Ron Isley"
print(len(multiRenames))
saveFile(idata=multiRenames, ifile="multiRenames.p")

# Get Single Results

In [ ]:
sorted(slimResults.items(), key=lambda pair: pair[1], reverse=False)
#for i, (artist, cnt) in enumerate(slimResults.most_common()):

In [ ]:
cutoff = 0.8
singleResult = {}
#for i, (artist, cnt) in enumerate(slimResults.most_common()):
for i, (artist, cnt) in enumerate(sorted(slimResults.items(), key=lambda pair: pair[1], reverse=False)):
    if i <= 1842:
        continue
    if i > 0 and i % 100 == 0 or i == 100:
        print("Passed",i,'/',len(slimResults))
    if cnt < 1:
        continue
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    if knownArtists.get(prevArtist) is not None:
        continue
    artistAlbums = set(list(fullResults[artist]["Songs"].keys()) + list(fullResults[artist]["Albums"].keys()))
    match = mulArts.getArtistNames(artist                                                     )
    if len(match) == 1:
        mdata = getMusicData("DiscArtist", list(match.keys())[0])
        if not isinstance(mdata, DataFrame):
            results = getBestArtistMatch(artist, artistAlbums, N=3, cutoff=cutoff)
            if results[2] is not None:
                if results[2] >= cutoff:
                    print(i,"/",len(slimResults),'  \t',artist,'\t',results)
                    singleResult[artist] = results
                    if len(singleResult) > 20:
                        break

In [ ]:
for k,v in singleResult.items():
    print("keep[\"{0}\"] = {1}  #{2}".format(k, v[0], v[1]))

In [ ]:
singleResults = {k: v[0] for k,v in singleResult.items()}
print("Found {0} single results".format(len(singleResults)))
print("Found {0} all results".format(len(knownArtists)))
knownArtists.update(singleResults)
print("Found {0} all results".format(len(knownArtists)))

In [ ]:
saveFile(idata=knownArtists, ifile="artistMap.p", debug=True)

## Manual Fixes

In [ ]:
keep["Wizkid"] = 3292269  #WizKid
keep["A. R. Rahman"] = 4459  #A.R. Rahman
keep["Pharell Williams"] = 90037  #Pharrell Williams
keep["Lookas"] = 3619379  #LooKas
keep["Lil Jon"] = 120307  #Lil' Jon
keep["Jay-z"] = 21742  #Jay-Z

keep["Jean Michel Jarre"] = 209415  #Jean-Michel Jarre
keep["Puppini Sisters"] = 663589  #The Puppini Sisters
keep["Hadouken"] = 744317  #Hadouken!
keep["Peter Doherty"] = 219403  #Pete Doherty
keep["Dani Konig"] = 95685  #Dani König
keep["X-centric Sound System"] = 177063  #Ex-Centric Sound System
keep["And You Will Know Us By The T"] = 80393  #And You Will Know Us By The Trail Of Dead
keep["Serena Maneesh"] = 563504  #Serena-Maneesh
keep["Potatoheadz"] = 38359  #Potatoheads
keep["Black Angels"] = 428135  #The Black Angels
keep["La Fiancee"] = 1603916  #La Fiancée
keep["Knut Anders Sorum"] = 713971  #Knut Anders Sørum

keep["Magic Numbers"] = 245226  #The Magic Numbers
keep["Melissa M"] = 952871  #Mélissa M
keep["Weird Al Yankovic"] = 259422  #"Weird Al" Yankovic
keep["Star Academy 6"] = 395756  #Star Academy
keep["Dj Boozywoozy"] = 39640  #DJ BoozyWoozy
keep["Lauri Tahka"] = 1236855  #Lauri Tähkä
keep["Finn Brothers"] = 377361  #The Finn Brothers
keep["Red Jumpsuit Apparatus"] = 746639  #The Red Jumpsuit Apparatus
keep["Katharine Mcphee"] = 544699  #Katharine McPhee
keep["Massive Tone"] = 51650  #Massive Töne
keep["Soul'd Out"] = 152449  #Soul'd OUT
keep["Djames Braun"] = 3024392  #Djämes Braun
keep["K-otic"] = 99126  #K-Otic
keep["Dr. Kucho"] = 171326  #Dr. Kucho!
keep["Sivert Hoyem"] = 226487  #Sivert Høyem
keep["DJ's @ Work"] = 27887  #DJs @ Work
keep["T.m.revolution"] = 1367682  #T.M.Revolution
keep["Suburban Tribe"] = 294619  #Sub-Urban Tribe
keep["Peer Gunt"] = 475654  #Peer Günt
keep["Acda En De Munnik"] = 283015  #Acda en de Munnik
keep["Susanne Sundfor"] = 747088  #Susanne Sundfør
keep["Zen Cafe"] = 264334  #Zen Café
keep["Lutricia Mcneal"] = 10896  #Lutricia McNeal
keep["Soul Decision"] = 725913  #soulDecision
keep["MC Anitta"] = 2807393  #Anitta
keep["Antonio Jose"] = 1661452  #Antonio José
keep["Miguel Angel Silva"] = 2374710  #Miguel Ángel Silva
keep["Giant Leap"] = 55439  #1 Giant Leap
keep["Stephanie McIntosh"] = 597377  #Stephanie Mcintosh
keep["Beautiful South"] = 171427  #The Beautiful South
keep["Mars Volta"] = 96218  #The Mars Volta
keep["India Arie"] = 410260  #India.Arie
keep["Sheepdogs"] = 1980121  #The Sheepdogs
keep["Bjorn Rosenstrom"] = 352899  #Björn Rosenström
keep["Pretty Reckless"] = 1773405  #The Pretty Reckless
keep["Wallflowers"] = 257714  #The Wallflowers
keep["Booming People"] = 440378  #The Booming People
keep["Tania Mara"] = 1614077  #Tânia Mara
keep["Zacharius Carl Group"] = 329135  #Zacharius Carls Group
keep["Daniel Lindstrom"] = 1465965  #Daniel Lindström
keep["Pigeon Detectives"] = 481793  #The Pigeon Detectives
keep["Last Shadow Puppets"] = 1099843  #The Last Shadow Puppets
keep["Brian Mckight"] = 97515  #Brian McKnight
keep["Mrs. Greenbird"] = 3134860  #Mrs Greenbird
keep["Ophelie Winter"] = 581072  #Ophélie Winter
keep["Les Deesses"] = 2392811  #Les Déesses
keep["Xander De Buisonje"] = 451325  #Xander De Buisonjé
keep["Partysquad"] = 523070  #The Partysquad
keep["Jose Gonzalez"] = 190821  #José González
keep["Ok Go"] = 219647  #OK Go
keep["Ladi6"] = 283432  #Ladi 6
keep["Sergio Mendes"] = 27986  #Sérgio Mendes
keep["Dorothee"] = 397164  #Dorothée
keep["Suvi Terasniska"] = 1032538  #Suvi Teräsniska
keep["Anna Jarvinen"] = 937842  #Anna Järvinen
keep["Kinderen Voor Kinderen"] = 354362  #Kinderen voor Kinderen
keep["Rippingtons"] = 555275  #The Rippingtons
keep["Flaming Lips"] = 67156  #The Flaming Lips
keep["William Balde"] = 1744946  #William Baldé
keep["Justice Collective"] = 3071540  #The Justice Collective
keep["Ultra Nate"] = 8099  #Ultra Naté
keep["Robin S"] = 66727  #Robin S.
keep["Trentemoller"] = 117914  #Trentemøller
keep["Lene Alexandra"] = 917923  #Lene Alexandra Øien
keep["Ready Set"] = 2183109  #The Ready Set
keep["Charlatans"] = 49023  #The Charlatans
keep["Maximo Park"] = 384412  #Maxïmo Park
keep["Mokobe"] = 392235  #Mokobé
keep["Paps 'n' Skar"] = 41801  #Paps N Skar
keep["ScHoolboy Q"] = 2353545  #Schoolboy Q
keep["Nelja Ruusua"] = 308352  #Neljä Ruusua
keep["Afro-dite"] = 296038  #Afro-Dite
keep["Breaks Co-op"] = 61311  #Breaks Co-Op
keep["Laurent Wery"] = 1310886  #Laurent Wéry
keep["Proclaimers"] = 274034  #The Proclaimers
keep["Courteeners"] = 971940  #The Courteeners
keep["No Tone"] = 841468  #No-Tone
keep["Mo Things Family"] = 174410  #Mo Thugs Family
keep["Sammy Deluxe"] = 88398  #Samy Deluxe
keep["LaCrim"] = 2707062  #Lacrim
keep["Trancelucent"] = 55384  #TranceLucent
keep["Luis Represas"] = 1237858  #Luís Represas
keep["Tatsurou Yamashita"] = 119485  #Tatsuro Yamashita
keep["Ana Tijoux"] = 469819  #Anita Tijoux
keep["Axe Bahia"] = 427149  #Axé Bahia
keep["Elakelaiset"] = 264335  #Eläkeläiset
keep["Turmion Katilot"] = 339414  #Turmion Kätilöt
keep["Jean Claude Ades"] = 66552  #Jean-Claude Ades
keep["Hammerfall"] = 287459  #HammerFall
keep["Hear'say"] = 312508  #Hear'Say
keep["Jack Penate"] = 747248  #Jack Peñate
keep["Jeremy Chatelain"] = 1107704  #Jérémy Chatelain
keep["Alex Max Band"] = 413545  #Alex Band
keep["Falsa Alarma"] = 1278412  #Falsalarma
keep["G-spott"] = 11679  #G-Spott
keep["Ray LaMontagne"] = 502002  #Ray Lamontagne
keep["Chris De Burgh"] = 151304  #Chris de Burgh
keep["Verve Pipe"] = 169539  #The Verve Pipe
keep["Plusch"] = 372645  #Plüsch
keep["Compagnie Creole"] = 528609  #La Compagnie Créole
keep["MC Fioti"] = 5865714  #Mc Fioti
keep["Sie7te"] = 1771803  #Sie7e
keep["Bastard Sons Of Dioniso"] = 2634291  #The Bastard Sons Of Dioniso
keep["Baby Blue Sound Crew"] = 1094228  #Baby Blue Soundcrew
keep["Stone Roses"] = 7298  #The Stone Roses
keep["DJ Matvey Emerson"] = 2126113  #Matvey Emerson
keep["Gerald De Palmas"] = 533880  #Gérald de Palmas
keep["Oli. P"] = 296389  #Oli.P
keep["Andre Hazes Jr."] = 4298003  #André Hazes Jr.
keep["Motorhead"] = 233658  #Motörhead
keep["Grateful Dead"] = 246650  #The Grateful Dead
keep["Liset Alea"] = 138872  #Lissette Alea
keep["Jennie Lofgren"] = 996876  #Jennie Löfgren
keep["Cardigans"] = 39900  #The Cardigans
keep["Cast Of Rent"] = 2531774  #The Cast Of Rent
keep["Saw Doctors"] = 544713  #The Saw Doctors
keep["Robert Delong"] = 1121732  #Robert DeLong
keep["Huun Huur Tu"] = 109547  #Huun-Huur-Tu
keep["Jari Sillanpaa"] = 713952  #Jari Sillanpää
keep["Hellacopters"] = 261426  #The Hellacopters
keep["Dan Backman"] = 1106992  #Dan Bäckman
keep["Artists Stand Up To Cancer"] = 1350696  #Just Stand Up To Cancer
keep["Mr.President"] = 41707  #Mr. President
keep["Loreena Mckennitt"] = 213366  #Loreena McKennitt
keep["Lashun Pace"] = 824951  #LaShun Pace
keep["Beach Boys"] = 70829  #The Beach Boys
keep["Allstars"] = 1009547  #TV Allstars
keep["Raveonettes"] = 200321  #The Raveonettes
keep["Body Rox"] = 269697  #Bodyrox
keep["Lazytown"] = 729388  #LazyTown
keep["Marie Laforet"] = 462548  #Marie Laforêt
keep["Dj Lhasa"] = 196037  #DJ Lhasa
keep["Robert Cray Band"] = 292478  #The Robert Cray Band
keep["Eppu Normaal"] = 381585  #Eppu Normaali
keep["Dj Norman"] = 23243  #DJ Norman
keep["B-Yentl"] = 2747976  #BYentl
keep["Jessica Folker"] = 28518  #Jessica Folcker
keep["Brolle"] = 572997  #Brolle JR
keep["The Cheetah Girls"] = 633276  #Cheetah Girls
keep["Pipettes"] = 360206  #The Pipettes
keep["John Dahlback"] = 20805  #John Dahlbäck
keep["HollySiz"] = 2082991  #Hollysiz
keep["Cassia Eller"] = 1105685  #Cássia Eller
keep["Marios Fragoulis"] = 328042  #Mario Frangoulis
keep["I Panta Nei"] = 1138210  #Panta Rei
keep["Sober"] = 946450  #Sôber
keep["Jarjestyshairio"] = 1931984  #Järjestyshäiriö
keep["Swingfly"] = 70432  #Swing-Fly
keep["Decemberists"] = 264812  #The Decemberists
keep["Jean Jacques Goldman"] = 307094  #Jean-Jacques Goldman
keep["Star Academy 7"] = 395756  #Star Academy
keep["Dj Chuckie"] = 135575  #DJ Chuckie
keep["Didrik Solli-tangen"] = 1818627  #Didrik Solli-Tangen
keep["Dj Goldfinger"] = 802185  #DJ Goldfinger
keep["Overtones"] = 1443138  #The Overtones
keep["Vaccines"] = 2029033  #The Vaccines
keep["Roshelle"] = 2321646  #Rochelle
keep["La Habitacion Roja"] = 393825  #La Habitación Roja
keep["Peter Lemarc"] = 266999  #Peter LeMarc
keep["Goldie Lookin' Chain"] = 184984  #Goldie Lookin Chain
keep["Smashing Pumpkins"] = 28970  #The Smashing Pumpkins
keep["Mighty Mighty Bosstones"] = 275193  #The Mighty Mighty Bosstones
keep["The Notorious B.I.G."] = 65049  #Notorious B.I.G.
keep["Chante Moore"] = 455230  #Chanté Moore
keep["Josh Gracin"] = 655983  #Joshua Gracin
keep["Franck Michael"] = 478419  #Frank Michael
keep["The Go-betweens"] = 83077  #The Go-Betweens
keep["Shakespears Sister"] = 30318  #Shakespear's Sister
keep["Piero Pelu"] = 662395  #Piero Pelù
keep["Los Delinquentes"] = 674561  #Los Delinqüentes
keep["St. Germain"] = 74  #St Germain
keep["Zuri West"] = 188532  #Züri West
keep["Bustafunk"] = 16272  #Busta Funk
keep["Lea Castel"] = 1153522  #Léa Castel
keep["De Lillos"] = 261173  #deLillos
keep["Dj The Wave"] = 241825  #DJ The Wave
keep["The Academy Is"] = 425277  #The Academy Is...
keep["Dj S.P.U.D."] = 42088  #DJ S.P.U.D.
keep["Dj Rebel"] = 209672  #DJ Rebel
keep["Susana Felix"] = 1333026  #Susana Félix
keep["Wildhearts"] = 293211  #The Wildhearts
keep["Altern8"] = 12846  #Altern 8
keep["Loredana Berte"] = 366682  #Loredana Bertè
keep["Bun-B"] = 185582  #Bun B
keep["Academia Operacion Triunfo"] = 1375332  #Academia Operación Triunfo
keep["Barr Brothers"] = 2470090  #The Barr Brothers
keep["Soulvation"] = 53390  #Soulvation*
keep["Ritmo Dynamic"] = 123374  #Ritmo-Dynamic
keep["Bleeders"] = 483312  #The Bleeders
keep["Hearsay"] = 312508  #Hear'Say
keep["Dj Shadow"] = 4478  #DJ Shadow
keep["Cooper Temple Clause"] = 138071  #The Cooper Temple Clause
keep["Maccabees"] = 499923  #The Maccabees
keep["Jon B."] = 20389  #Jon B
keep["North American Halloween Prevention Initiative"] = 1039212  #North American Hallowe'en Prevention Initiative
keep["Cerena"] = 1894383  #Cérena
keep["Ze Pequeno"] = 4613736  #Ze Pequeño
keep["Les Muscles"] = 297430  #Les Musclés
keep["Teki Latex"] = 247176  #Tekilatex
keep["Tommy February6"] = 1283490  #Tommy february6
keep["Tommy Februaryo"] = 1283490  #Tommy february6
keep["Lovefreekz"] = 208128  #The Lovefreekz
keep["Upper Room"] = 487039  #The Upper Room
keep["Avett Brothers"] = 824244  #The Avett Brothers
keep["Legiao Urbana"] = 264082  #Legião Urbana
keep["Whitlams"] = 254486  #The Whitlams
keep["Jo Jo"] = 306427  #JoJo
keep["Fundacion Tony Manero"] = 194603  #Fundación Tony Manero
keep["Ali B."] = 234355  #Ali B
keep["De Toppers"] = 602723  #Toppers
keep["Motorhomes"] = 252568  #The Motorhomes
keep["Di Leva"] = 75662  #Di leva
keep["Timo Raisanen"] = 277925  #Timo Räisänen
keep["Sebastien Tellier"] = 2280  #Sébastien Tellier
keep["Albatraoz"] = 1183850  #Alcatraz
keep["Roisin Murphy"] = 455520  #Róisín Murphy
keep["Ha Rule"] = 51369  #Ja Rule
keep["Pepper's Ghost"] = 2364386  #Peppers Ghost
keep["Waterboys"] = 125174  #The Waterboys
keep["Georges-alain Jones"] = 1940009  #Georges-Alain Jones
keep["Neg'marrons"] = 219331  #Neg'Marrons
keep["Kaleidoscopio"] = 166678  #Kaleidoscópio
keep["alt-J"] = 2830806  #Alt-J
keep["Tommy February"] = 1283490  #Tommy february6
keep["Disco Boys"] = 196758  #The Disco Boys
keep["Natalia Jimenez"] = 2130134  #Natalia Jiménez
keep["MadMan"] = 2668959  #Madman
keep["Aitana Ocana"] = 6371628  #Aitana Ocaña
keep["Polyphonic Spree"] = 79023  #The Polyphonic Spree
keep["John Mellencamp"] = 237890  #John Cougar Mellencamp
keep["White Tie Affair"] = 1943163  #The White Tie Affair
keep["Bon Garcon"] = 458392  #Bon Garçon
keep["Fu-tourist"] = 34401  #Fu-Tourist
keep["Kapteeni A-ni"] = 85882  #Kapteeni Ä-ni
keep["Ron Van Den Beuken"] = 90120  #Ron van den Beuken
keep["For My Pain"] = 619891  #For My Pain...
keep["Jack McManus"] = 808532  #Jack Mcmanus
keep["Radio Dept."] = 238640  #The Radio Dept.
keep["Elin Sigvardsson"] = 1063935  #Elin Ruth Sigvardsson
keep["Dj Felli Fel"] = 227465  #Felli Fel
keep["New Deal"] = 44720  #The New Deal
keep["Camera Cafe"] = 1645853  #Caméra Café
keep["Ben L`Ocle Soul"] = 1689692  #Ben L'Oncle Soul
keep["Veronique Sanson"] = 394617  #Véronique Sanson
keep["Dj Schwede"] = 41815  #DJ Schwede
keep["Beam Vs. Cyrus"] = 19442  #Beam vs. Cyrus
keep["Dj Tocadisco"] = 424389  #Tocadisco
keep["De Vrienden Van Meneer Konijn"] = 4591921  #Vrienden Van Meneer Konijn
keep["4 Taste"] = 325990  #Taste
keep["Shaka Labbits"] = 1829370  #Shakalabbits
keep["Elio E Le Storie Tes"] = 191166  #Elio E Le Storie Tese
keep["Paco De Lucia"] = 20184  #Paco De Lucía
keep["Format B"] = 263420  #Format: B
keep["Oceanlab"] = 427638  #OceanLab
keep["Motley Crue"] = 94068  #Mötley Crüe
keep["Janelle Monae"] = 445868  #Janelle Monáe
keep["Civil Wars"] = 2001094  #The Civil Wars
keep["Moneybagg Yo"] = 5197378  #MoneyBagg Yo
keep["Caleidoscopio"] = 166678  #Kaleidoscópio
keep["Zeljko Joksimovic"] = 473482  #Željko Joksimović
keep["Mousee T"] = 8803  #Mousse T.
keep["Polo Montanes"] = 483555  #Polo Montañez
keep["Annette Artani"] = 1466811  #Annet Artani
keep["Payo Malo"] = 613450  #El Payo Malo
keep["To Die For"] = 239041  #To-Die-For
keep["Dj Jurgen"] = 17260  #DJ Jurgen
keep["Modesha"] = 128255  #Nodesha
keep["Latin Kings"] = 158482  #The Latin Kings
keep["Dead Weather"] = 1391789  #The Dead Weather
keep["High School Musical Cast"] = 673607  #The High School Musical Cast
keep["Dj Mark Farina"] = 4710  #Mark Farina
keep["Sandrine Francois"] = 1446111  #Sandrine François
keep["Gregori Baquet"] = 568948  #Grégori Baquet
keep["Superheavy"] = 2418975  #SuperHeavy
keep["Deborah De Corral"] = 2035021  #Deborah de Corral
keep["Brilliant Green"] = 296909  #The Brilliant Green
keep["Laith Al-deen"] = 341824  #Laith Al-Deen
keep["Audiobullys"] = 50016  #Audio Bullys
keep["Mr Redz"] = 21964  #Mr. Reds
keep["Eye Opener"] = 138167  #Eyeopener
keep["Kraftklub"] = 2028458  #KraftKlub
keep["Human League"] = 10383  #The Human League
keep["Lost Brothers"] = 158835  #The Lost Brothers
keep["Young Knives"] = 470191  #The Young Knives
keep["Get Cape Wear Cape Fly"] = 747070  #Get Cape. Wear Cape. Fly
keep["Starting Line"] = 355465  #The Starting Line
keep["The-dream"] = 1008036  #The-Dream
keep["Piano Guys"] = 3507956  #The Piano Guys
keep["Beyonce"] = 52835  #Beyoncé
keep["Amity Affliction"] = 2446213  #The Amity Affliction
keep["A$AP Mob"] = 2898554  #ASAP Mob
keep["Kelly K"] = 970192  #Kelly Key
keep["Blizzard Brothers Inc"] = 41806  #Blizzard Brothers
keep["Jade Macrae"] = 197656  #Jade MacRae
keep["Herman Dune"] = 264602  #Herman Düne
keep["Alex  Britti"] = 653853  #Alex Britti
keep["Alfred Garcia"] = 6371630  #Alfred García
keep["Rena Dif"] = 233790  #René Dif
keep["Flaming Sideburns"] = 491667  #The Flaming Sideburns
keep["JS-16"] = 35833  #JS16
keep["Lisa Left Eye Lopes"] = 110359  #Lisa "Left Eye" Lopes
keep["Rosemary' Sons"] = 285835  #Rosemary's Sons
keep["K-klass"] = 36624  #K-Klass
keep["Drumatic Twins"] = 14784  #Drumattic Twins
keep["Crashdiet"] = 825015  #Crashdïet
keep["Anne-Lie Ryde"] = 264170  #Anne-Lie Rydé
keep["Alex Swing Oskars Sings!"] = 1442099  #Alex Swings Oscar Sings!
keep["Adolphson Falk"] = 119104  #Adolphson-Falk
keep["Run DMC"] = 219213  #Run-DMC
keep["Pj Harvey"] = 36052  #PJ Harvey
keep["Journey South"] = 325103  #Joe South
keep["Lil Scrappy"] = 239305  #Lil' Scrappy
keep["Breeders"] = 39778  #The Breeders
keep["Huis Anubis"] = 2797437  #Het Huis Anubis
keep["Avalanches"] = 9130  #The Avalanches
keep["Dj F.E.X."] = 43804  #DJ F.E.X
keep["Kc Da Rookee"] = 220984  #KC Da Rookee
keep["N Trance"] = 11001  #N-Trance
keep["Trooper Da Doon"] = 66144  #Trooper Da Don
keep["Divine Comedy"] = 27933  #The Divine Comedy
keep["Blumchen"] = 20156  #Blümchen
keep["Dt8 Project"] = 34061  #DT8 Project
keep["Dead 60s"] = 260915  #The Dead 60s
keep["Duke Spirit"] = 275448  #The Duke Spirit
keep["Airborne Toxic Event"] = 1311029  #The Airborne Toxic Event
keep["Big Pink"] = 1337426  #The Big Pink
keep["Boyz In Da Hood"] = 336836  #Boyz N Da Hood
keep["Devil Wears Prada"] = 989173  #The Devil Wears Prada
keep["Sixx: A.M."] = 895454  #Sixx:A.M.
keep["Dirty Heads"] = 2089008  #The Dirty Heads
keep["Mac DeMarco"] = 2568722  #Mac Demarco
keep["Mauricio Manieri"] = 1563247  #Maurício Manieri
keep["Butterfly Effect"] = 441749  #The Butterfly Effect
keep["L'aura"] = 880722  #L'Aura
keep["Fabula"] = 210657  #Jabula
keep["Dj Joe K"] = 225215  #DJ Joe K.
keep["Rasmus Nohr"] = 396235  #Rasmus Nøhr
keep["Valkyrians"] = 538336  #The Valkyrians
keep["Dir En Grey"] = 348163  #Dir en grey
keep["Rock'a'trench"] = 2981100  #Rock'A'Trench
keep["Wolfe Tones"] = 1219696  #The Wolfe Tones
keep["Andre Van Duin"] = 367055  #André van Duin
keep["M Hederos M Hellberg"] = 474515  #Hederos & Hellberg
keep["Christer Sjogren"] = 656866  #Christer Sjögren
keep["Feelstyle"] = 266896  #Tha Feelstyle
keep["Beta Band"] = 3847  #The Beta Band
keep["Angelis"] = 251965  #Angelfish
keep["Dr Hook"] = 206325  #Dr. Hook
keep["Royal Scots Dragoon Guards"] = 595755  #The Royal Scots Dragoon Guards
keep["Hold Steady"] = 491197  #The Hold Steady
keep["SheDaisy"] = 1479264  #Shedaisy
keep["Gregoire"] = 1407221  #Grégoire
keep["Tragically Hip"] = 267249  #The Tragically Hip
keep["Dj Molella"] = 36544  #Molella
keep["Facteur X"] = 334431  #Factor X
keep["Che Nelle"] = 1055693  #Che'Nelle
keep["Jerome Echenoz"] = 695381  #Jérôme Echenoz
keep["Svein Ostvik"] = 3781616  #Svein Østvik
keep["Dj Taylor"] = 16567  #DJ Taylor
keep["Wiener Sangerknaben"] = 533488  #Die Wiener Sängerknaben
keep["Zoe Straub"] = 4767549  #Zoë Straub
keep["David Latour "] = 1189620  #David Latour
keep["Finger Tips"] = 1663670  #Fingertips
keep["Kelly Family"] = 319276  #The Kelly Family
keep["Jaime Cullum"] = 194401  #Jamie Cullum
keep["Queensryche"] = 255363  #Queensrÿche
keep["Black Crowes"] = 262691  #The Black Crowes
keep["David Crowder Band"] = 413886  #David Crowder*Band
keep["Gaslight Anthem"] = 1167086  #The Gaslight Anthem
keep["BrockHampton"] = 4581123  #Brockhampton
keep["Getaway Plan"] = 1230298  #The Getaway Plan
keep["Ener G"] = 1660398  #Ener.G
keep["C- Bool"] = 251008  #C-Bool
keep["Dublex INC."] = 3762  #Dublex Inc.
keep["R.E.G. Project"] = 164534  #The R.E.G. Project
keep["Niccolo Fabi"] = 329588  #Niccolò Fabi
keep["Jakkata"] = 3442  #Jakatta
keep["Rald Schmitz"] = 1404551  #Ralf Schmitz
keep["Bikstok Rogsystem"] = 272410  #Bikstok Røgsystem
keep["Sinead O'Connor"] = 42895  #Sinéad O'Connor
keep["Dj Slow"] = 55897  #DJ Slow
keep["Buck-tick"] = 69223  #Buck-Tick
keep["Mahala Rai Banda"] = 1208659  #Mahala Raï Banda
keep["SunStroke Project"] = 1818637  #Sunstroke Project
keep["Gary Clark Jr"] = 2699369  #Gary Clark Jr.
keep["Royal Guardsmen"] = 290319  #The Royal Guardsmen
keep["Temptations"] = 28332  #The Temptations
keep["Searchers"] = 277352  #The Searchers
keep["New Seekers"] = 93841  #The New Seekers
keep["Counting Crow"] = 262643  #Counting Crows
keep["Louise Attague"] = 374074  #Louise Attaque
keep["Osborne Brothers"] = 1088676  #The Osborne Brothers
keep["Roman Flugel"] = 13111  #Roman Flügel
keep["Rubettes"] = 240902  #The Rubettes
keep["Bart Kaell"] = 446710  #Bart Kaëll
keep["Slongs Dievanongs "] = 4435213  #Slongs Dievanongs
keep["Ez Special"] = 148110  #EZ Special
keep["Reelists"] = 154054  #The Reelists
keep["King Gidra"] = 385694  #King Giddra
keep["Zoe Birkett"] = 320018  #Zoë Birkett
keep["D!-Nation"] = 1599356  #D!Nation
keep["Von Bondies"] = 313529  #The Von Bondies
keep["Weather Girls"] = 80134  #The Weather Girls
keep["Levellers"] = 23672  #The Levellers
keep["Mj Cole"] = 1515  #MJ Cole
keep["Paddingtons"] = 337981  #The Paddingtons
keep["Long Blondes"] = 245214  #The Long Blondes
keep["Sunshine Underground"] = 361710  #The Sunshine Underground
keep["Unklejam"] = 774556  #UnkleJam
keep["Dave Clark Five"] = 329252  #The Dave Clark Five
keep["Boney M"] = 235979  #Boney M.
keep["Trustcompany"] = 279197  #Trust Company
keep["K`Jon"] = 211210  #K'Jon
keep["New Pornographers"] = 363443  #The New Pornographers
keep["Grace Vanderwaal"] = 5436370  #Grace VanderWaal
keep["Lil Peep"] = 5378070  #Lil' Peep
keep["Superjesus"] = 273378  #The Superjesus
keep["Dissociatives"] = 233572  #The Dissociatives
keep["Sleepy Jackson"] = 138068  #The Sleepy Jackson
keep["McClymonts"] = 2376183  #The McClymonts
keep["Jezabels"] = 1704414  #The Jezabels
keep["Raining Pleausure"] = 637513  #Raining Pleasure
keep["Loredana Berte'"] = 366682  #Loredana Bertè
keep["Claduio Baglioni"] = 638411  #Claudio Baglioni
keep["Papa Levante"] = 388519  #Papá Levante
keep["Dj Marta"] = 198294  #DJ Marta
keep["Mendonca Do Rio"] = 783996  #Mendonça Do Rio
keep["Epila"] = 1608899  #Epilä
keep["Man-Eating Tree"] = 2289900  #The Man-Eating Tree
keep["Jon Norgaard"] = 845262  #Jon Nørgaard
keep["Black League"] = 404409  #The Black League
keep["Pate Mustajarvi"] = 477874  #Pate Mustajärvi
keep["Brand New Heavies"] = 1396  #The Brand New Heavies
keep["Erik E"] = 9647  #Erick E
keep["Dj Mobster"] = 943953  #DJ Mobster
keep["Frank Ti-aya"] = 583397  #Frank Ti-Aya
keep["Osten Med Resten"] = 704765  #Östen Med Resten
keep["Ape"] = 404433  #Apse
keep["Tough Alliance"] = 305005  #The Tough Alliance
keep["Kristet Utseende"] = 360021  #The Kristet Utseende
keep["Kalomoira"] = 2270989  #Kalomira

In [ ]:
keep["Ms Dynamite"] = 24059  #Ms. Dynamite
keep["Rene Froger"] = 283021  #René Froger
keep["VanVelzen"] = 702319  #Vanvelzen
keep["Fratellis"] = 472288  #The Fratellis
keep["Andre Hazes"] = 282287  #André Hazes
keep["Kat-tun"] = 2884266  #Kat-Tun
keep["Jean Roch"] = 1368909  #Jean-Roch
keep["TikTak"] = 383787  #Tiktak
keep["L.E.J."] = 4777768  #L.E.J
keep["Pablo Lopez"] = 3465939  #Pablo López
keep["Agnetha Faltskog"] = 149038  #Agnetha Fältskog
keep["Kapten Rod"] = 1104466  #Kapten Röd
keep["BBmak"] = 231074  #BBMak
keep["Dj Jose"] = 53783  #DJ Jose
keep["Gigi D'alessio"] = 839027  #Gigi D'Alessio
keep["Terasbetoni"] = 333752  #Teräsbetoni
keep["Kim-lian"] = 323824  #Kim-Lian
keep["Jose Feliciano"] = 465889  #José Feliciano
keep["Asteroids Galaxy Tour"] = 865897  #The Asteroids Galaxy Tour
keep["Strumbellas"] = 3705444  #The Strumbellas
keep["Pep`s"] = 701116  #Pep's
keep["Gue Pequeno"] = 1216738  #Guè Pequeno
keep["Dead By April"] = 1477009  #Dead by April
keep["Luca Hanni"] = 2779963  #Luca Hänni
keep["Noir Desir"] = 85007  #Noir Désir
keep["Jean Pascal"] = 614758  #Jean-Pascal
keep["Herbert Groenemeyer"] = 163850  #Herbert Grönemeyer
keep["Nina Pastori"] = 925146  #Niña Pastori
keep["Saddle Club"] = 2956368  #The Saddle Club
keep["Pariisin Kevat"] = 1984505  #Pariisin Kevät
keep["Barbara Streisand"] = 53248  #Barbra Streisand
keep["Bohse Onkelz"] = 262898  #Böhse Onkelz
keep["Manolo Garcia"] = 705504  #Manolo García
keep["Ordinary Boys"] = 366147  #The Ordinary Boys
keep["Dj Tatana"] = 13864  #DJ Tatana
keep["Peter Joback"] = 343362  #Peter Jöback
keep["Loic Nottet"] = 4355587  #Loïc Nottet
keep["Sander Van Doorn"] = 183267  #Sander van Doorn
keep["Youngbloodz"] = 209479  #YoungBloodZ
keep["A$AP Ferg"] = 2503395  #ASAP Ferg
keep["Benassi Bros"] = 171187  #Benassi Bros.
keep["Watermat"] = 3856896  #Watermät
keep["Asian Kung-fu Generation"] = 289562  #Asian Kung-Fu Generation
keep["Hi-Tack"] = 364150  #Hi_Tack
keep["Hubert Von Goisern"] = 446821  #Hubert von Goisern
keep["Lee Dewyze"] = 2016926  #Lee DeWyze
keep["Sofia Essaidi"] = 1400771  #Sofia Essaïdi
keep["Elodie Frege"] = 958245  #Elodie Frégé
keep["Jurgen Drews"] = 109458  #Jürgen Drews
keep["Jorge Vercilo"] = 1609339  #Jorge Vercillo
keep["Futureheads"] = 132795  #The Futureheads
keep["Jim Stark"] = 273036  #Jim Stärk
keep["Andre Sardet"] = 1324640  #André Sardet
keep["Mago De Oz"] = 691599  #Mägo De Oz
keep["Kapasiteettiyksikko"] = 264342  #Kapasiteettiyksikkö

In [ ]:
keys = ["Click Five", "Gospellers", "Temper Trap", "Cranberries", "Matthew Good Band", "Esmee Denters", "David Demaria",
        "Hoosiers", "L'Arc-en-Ciel", "Dj Aligator Project", "Diana Degarmo", "Adelen", "Madden Brothers", "Grupo Revelacao",
        "Brolle Jr.", "Beatfreakz", "Star Academy 2"]
for key in keys:
    keep[key] = singleResult[key][0]

In [ ]:
keys = ["Potbelleez", "Los Autenticos Decadentes", "Gunther", "Star Academy 5", "Pedro Capo", "Dandy Warhols",
        "Postal Service", "Tea Party", "B 3", "Dj Tomekk", "M-kids", "Waldo`s People", "Lil Kleine", "Laura Narhi", "TopGunn", 
        "Joaquin Sabina", "Raconteurs", "Monica Naranjo", "Patrick Sebastien", "Molly Sanden", "Miguel Bose"]
for key in keys:
    keep[key] = singleResult[key][0]

In [ ]:
keys = ['Petri Nygard', 'Blackbear', 'Den Svenska Bjornstammen', 'Baseballs', 'Sandra Van Nieuwland', 'Ian Carey Project',
        "BossHoss", "Max Gazze", "MoTrip", "Les Enfoires", "Lil Bow Wow", "Frero Delavega", "Kumi Kouda", "Bjork", 
        "RedFoo", "Star Academy 4", "John Butler Trio", "Gregory Lemarchal"]
for key in keys:
    keep[key] = singleResult[key][0]

In [ ]:
keys = ['AronChupa', 'Sexion D`Assaut', 'Die Arzte', "Bro'sis", "Maneskin", "Royksopp", "Opposites", "OpShop", 
        "Jennifer Pena", "Paul Van Dyk", "La 5ta Estacion", "Supermen Lovers", "Keshia Chante", "Special D",
        "Kinki Kids", "Neighbourhood", "No Te Va A Gustar", "Ll Cool J", "Jeroen Van Der Boom", "Star Academy 3",
        "Miguel Angel Munoz", "Tito El Bambino", "Andres Calamaro", "A$AP Rocky", "Boogie Pimps"]
for key in keys:
    keep[key] = singleResult[key][0]

In [ ]:
keys = ['DJ Otzi', 'DJ Bobo', 'R.I.O.', 'Chimene Badi', 'Underdog Project', 'Thalia', 'Mr Children', 'Lonely Island', 'Tragedie',
        'Burhan G', 'Mans Zelmerlow', 'Christophe Mae', 'Marco Antonio Solis', 'Sohne Mannheims', 'US5', 'Lars Winnerback',
        'Alejandro Fernandez', 'Mis-teeq', 'Gestort Aber Geil', 'Herbert Gronemeyer', "Shy'M", "Hakan Hellstrom",
        "Keen`V", "Ting Tings", "Frankie J", "K 3", "Common Linnets", "Coeur De Pirate", "Julien Dore", 'Dani Martin', 'Di-rect']
for key in keys:
    keep[key] = singleResult[key][0]

In [ ]:
keys = ['T.A.T.U.', 'Outkast', 'Mr Probz', 'Michael Buble', 'B.o.B', 'Michel Telo', 'Pablo Alboran', 'O-zone', 'Amy Macdonald',
        'Lumineers', 'DJ Tiesto', 'Chainsmokers','MadCon', 'Mylene Farmer', 'XXXTentacion', 'Dj Sammy', 'Sophie Ellis Bextor',
        'Veronicas', 'In Grid', 'Chemical Brothers', 'K-Maro', 'Collectif Metisse', 'Matt Pokora', 'White Stripes', 'Bebe Lilly',
        'Calling', 'Saturdays', 'Christina Sturmer', 'All-American Rejects', 'Dan Balan', 'J Balvin', 'Elena Paparizou', 
        'Nadiya', 'Blink 182', 'Rene La Taupe', 'DonkeyBoy', 'Rolling Stones', 'Sigur Ros', 'Israel Kamakawiwoole', 'Armin Van Buuren']
for key in keys:
    keep[key] = singleResult[key][0]

In [ ]:
print(keep)
saveFile(idata=keep, ifile="known.p", debug=True)

In [ ]:
keep = getFile("known.p")
saveFile(idata=keep, ifile="singleRenames.p")
print(keep)

In [ ]:
multiGet  = {}
singleGet = {}
matchGet  = {}


for i, (artist, cnt) in enumerate(slimResults.most_common()):    
    artistAlbums = set(list(fullResults[artist]["Songs"].keys()) + list(fullResults[artist]["Albums"].keys()))
    matches = mulArts.getArtistNames(artist)
    for match in matches.keys():
        if multiGet.get(match) is not None:
            continue
        if singleGet.get(match) is not None:
            continue
        if matchGet.get(match) is not None:
            continue
        mdata = getMusicData("DiscArtist", match)
        if not isinstance(mdata, DataFrame):
            if len(matches) == 1:
                singleGet[match] = artistAlbums
                print("{0: <20}{1: <30}{2: <30}{3}\tSingle".format("{0} / {1}".format(i,len(slimResults)), match, artist, cnt))
            else:
                multiGet[match] = artistAlbums
                print("{0: <20}{1: <30}{2: <30}{3}\tMulti".format("{0} / {1}".format(i,len(slimResults)), match, artist, cnt))
        else:
            if mdata.shape[0] == 1:
                continue
            else:
                matchGet[match] = artistAlbums
                print("{0: <20}{1: <30}{2: <30}{3}\tMatch".format("{0} / {1}".format(i,len(slimResults)), match, artist, cnt))
    if i > 1000:
        break

In [ ]:
saveFile(idata=multiGet, ifile="multiToGet.p")
saveFile(idata=singleGet, ifile="singleGet.p")
saveFile(idata=matchGet, ifile="matchGet.p")

In [ ]:
for artistName, artistResults in multiGet.items():
    results = getBestArtistMatch(artistName, artistResults, N=10, cutoff=0.6)
    print(artistName,'\t',results)

In [ ]:
%load_ext autoreload
%autoreload

from artists import artists
disc = discogs()
arts = artists(disc)
for artistName in singleGet.keys():
    print("===========>",artistName)
    arts.searchDiscogForArtist(artistName)

In [ ]:
from collections import Counter
x = Counter({'a': 0, 'b': 0.5, 'c': 0.75})
x.most_common(1)

In [ ]:
getBestArtistMatch("Beyonce", {'Crazy In Love', 'Deja Vu', 'Drunk In Love'})

In [ ]:
ratVal = 0.6
if len(myMusicNameIDMap) > 0:
    ratVal = 0.3
artistIDMap = {}
for artistName,artistSlimData in multiMap.items():
    print("\n","="*50)
    print("ArtistName: {0}".format(artistName))
    print("   Albums: {0}".format(len(artistSlimData["Albums"])))

    myAlbumNames = []
    for album in artistSlimData["Albums"]:
        myAlbumName = album.split("/{0}/".format(artistName))[-1]
        #print("     {0: <15}{1: <10}{2}".format("", "", myAlbumName))
        myAlbumNames.append(myAlbumName)
        
    for idx,row in artistSlimData["DB"].iterrows():
        artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
        artistAlbums     = artistAlbumsData["Albums"]
        print("   Match: {0}  [{1}]".format(row["DiscArtist"], row["Name"]))
        if isinstance(artistAlbums, dict):
            for albumType, albumTypeData in artistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    for myAlbumName in myAlbumNames:
                        s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                        ratio = s.ratio()
                        if ratio > 0.6:
                            print("     {0: <15}{1: <10}{2: <8}{3}".format(albumType, albumID, round(ratio,2), dbAlbumName))
                            artistIDMap[artistName] = [idx, row["Name"]]

In [ ]:
daytype = {"USA Albums": ["SAT", "%Y-%m-%d"],
           "USA Singles Top 40": ["SAT", "%Y-%m-%d"]}
daytype = {"UK Singles Top 40": ["SAT", "%Y-%m-%d"]}
daytype = {"Top40-Charts.com Web Top 100": ["SAT", "%Y-%m-%d"]}
daytype = {"Airplay World Official Top 100": ["SAT", "%Y-%m-%d"]}

#daytype = {"USA Singles Top 40": ["SAT", "%Y-%m-%d"]}

from pandas import Timestamp
def getDates(daytype, cname):
    dates = []
    if cname == "USA Singles Top 40":
        dates1 = date_range(start='1997-07-07', end="2000-04-01", freq="W-MON").strftime("%Y-%m-%d")
        dates2 = date_range(start="2000-04-01", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)
    if cname == "UK Singles Top 40":
        dates1 = date_range(start="2002-12-08", end='2011-11-12', freq="W-SUN").strftime("%Y-%m-%d")
        dates2 = date_range(start="2011-11-12", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)        
    if cname == "Top40-Charts.com Web Top 100":
        dates1 = date_range(start="2002-10-14", end='2005-12-26', freq="W-MON").strftime("%Y-%m-%d")
        dates2 = date_range(start="2009-02-28", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)
    if cname == "Airplay World Official Top 100":
        dates1 = date_range(start="2002-01-07", end='2002-01-21', freq="W-MON").strftime("%Y-%m-%d")
        dates2 = date_range(start="2002-02-02", end=Timestamp.today(), freq="W-SAT").strftime("%Y-%m-%d")
        dates = list(dates1) + list(dates2)
    return dates

In [ ]:
def showDates(year, day):
    return date_range(start=str(year), end=str(int(year)+1), freq='W-{0}'.format(day))
    
showDates(2011, 'SUN')

In [ ]:
from time import sleep
for cid,cname in charts.items():
    if daytype.get(cname) is None:
        continue
    dates = getDates(daytype, cname)
    
    ##savedir  = join(basedir, "data", "top40", "starters")
    ##savename = join(savedir, "{0}.p".format(cname.replace("/", " ")))
    ##starter  = getHTML(savename)
    #dates    = getDates(starter)
    
    for idts,datename in enumerate(dates):
        
        if idts > 2000:
            break
        url      = "https://top40-charts.com/chart.php?cid={0}&date={1}".format(cid, datename)
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,}
    
        savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
        mkDir(savedir)

        savename = setFile(savedir, "{0}.p".format(datename))
        if isFile(savename):
            continue

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        print("URL ---> {0}".format(url))
        print(idts,'/',len(dates),"\tSaving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(3)
        

#list(all_sundays(2001))

In [ ]:
from time import sleep
for cid,cname in charts.items():
    if daytype.get(cname) is None:
        continue    
    dates = getDates(daytype, cname)
    
    for idts,datename in enumerate(dates):
        savedir  = join(basedir, "data", "top40", cname.replace("/", " "))
        savename = setFile(savedir, "{0}.p".format(datename))

        chartData = getHTML(savename)
        results = getChartData(chartData, debug=False)
        if len(results) == 0:
            print(cname,'\t\t',datename,'\t',len(results)," <<-------")
        else:
            print(cname,'\t\t',datename,'\t',len(results))

# Process Chart Data

In [ ]:
savedir  = join(basedir, "data", "top40")
dirnames = [x for x in findDirs(savedir) if x.endswith("starters") == False]
for dirname in dirnames:
    files = findExt(dirname, ext="*")
    for ifile in files:
        src = ifile
        dst = "{0}.p".format(ifile)
        moveFile(src, dst)


In [ ]:
savedir  = join(basedir, "data", "top40")
dirnames = [x for x in findDirs(savedir) if x.endswith("starters") == False]
for dirname in dirnames:
    files = findExt(dirname, ext="*.p")
    for ifile in files:
        chartData = getHTML(ifile)
        break

In [ ]:
chartData

In [ ]:
chartData = getHTML("/Users/tgadfort/Documents/code/charts/data/top40/test.html")

In [ ]:
def getChartData(chartData):
    artists = []
    titles = []
    
    trs = chartData.findAll("tr", {"class": "latc_song"})    
    for xs in [x.findAll("a", {"title": "View song details"}) for x in trs]:
        titles.append([x.text for x in xs if x.find("img") == None][0])

    for xs in [x.findAll("a", {"style": "text-decoration: none; "}) for x in trs]:
        artists.append(xs[0].text)
    
    retval = dict(zip(artists, titles))
    return retval

In [ ]:

len(chartData.findAll('table'))

In [ ]:
from searchUtils import findExt

In [ ]:
files = findExt("data/top40/World Singles Official Top 100/", ext=".p")

In [ ]:
for ifile in files[:5]:
    print(ifile)
    chartData = getHTML(ifile)
    
    for it,x in enumerate(chartData.findAll("table")):
        trs = x.findAll("tr", {"class": "latc_song"})
        for itr,tr in enumerate(trs):
            tds = tr.findAll("td")
            for itd,td in enumerate(tds):
                hrefs = td.findAll("a")
                for ihref,href in enumerate(hrefs):
                    print(it,'\t',itr,'\t',itd,'\t',ihref,'\t',href.text)

In [ ]:
for it,x in enumerate(chartData.findAll("table")):
    print(it,len(x))

In [ ]:
chartData.findAll("table")[8]

In [ ]:
files = findExt("data/top40/World Singles Official Top 100/", ext=".p")

In [ ]:
files[0]

In [ ]:
getHTML(files[0])

In [ ]:
def all_sundays(year):
# January 1st of the given year
       dt = date(year, 1, 1)
# First Sunday of the given year       
       dt += timedelta(days = 6 - dt.weekday())  
       while dt.year == year:
            yield dt
            dt += timedelta(days = 7)
            
for s in all_sundays(2020):
    m = s.month
    d = s.day
    y = s.year
    print(s.strftime("%d-%m-%Y"))
    print(type(s))

In [ ]:
d = "2002-01-07"

In [ ]:
files = findExt("data/top40/old/World Singles Official Top 100/", ext=".p")

In [ ]:
bs = getHTML(files[0])

In [ ]:
bs.find("Iglesias")

In [ ]:
url = "https://top40-charts.com/chart.php?cid=35&date=2002-01-07"

In [ ]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}

savename = setFile(savedir, "mytest.p")

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need

print(idts,'/',len(dates),"\tSaving {0}".format(savename))
saveJoblib(data=data, filename=savename, compress=True)

In [ ]:
str(data).find("Enya")

In [ ]:
N   = 28
obs = 0.5
r   = obs
Nmc = 250000

In [ ]:
import random
from numpy import corrcoef
from pandas import Series

In [ ]:
corrs = []
for i in range(Nmc):
    x1 = [random.random() for x in range(N)]
    x2 = [random.random() for x in range(N)]
    corrs.append(corrcoef(x1,x2)[0][1])

In [ ]:
sc = Series(corrs)
sc.hist(bins=100)
Nabove = len(sc[sc >= obs])
pvalue = Nabove / Nmc
import scipy.stats as st
print("{0} / {1}".format(Nabove,Nmc))
print("N --> {0}".format(N))
print("r --> {0}".format(r))

tscore = r*sqrt(N-2)/sqrt(1-r**2)
print("t --> {0}".format(t))
p  = 1 - stats.t.cdf(tscore,df=N-2)
print("p --> {0}".format(p))


from scipy.stats import norm
for x in linspace(0,5,500):
    p = 2*(1 - norm.cdf(x))
    if p <= pvalue:
        print("p-value  --> {0}".format(pvalue))
        print("# of std --> {0}".format(x))
        break

In [ ]:
from numpy import linspace
rcorrs = {}
for rho in linspace(-1,1,100):
    rcorrs[rho] = rho*sqrt(N-2)/sqrt(1-rho**2)
Series(rcorrs).plot()